# Analisi statistica intervalli di ricorrenza extreme returns

Questo notebook contiene le analisi statistiche fatte per il paper.

Il flusso è il seguente:

- [x] utilizzo del dataset *S&P500* con la massima ampiezza storica disponibile (2005 - 2018)
- [x] calcolo dei log returns
- [x] divisione in training e testing set prima di calcolare la volatilità
- [x] calcolo della volatilità per tutte le stocks, sul *training set*:
    - [x] come deviazione standard del prezzo di chiusura
    - [x] come $\beta$ coefficient
- [x] selezione di due stocks, quelle con la minima e la massima volatilità in tutto il periodo considerato
- [x] plot degli intervalli di ricorrenza per il Dow Jones, con metodi diversi:
    - [x] quantile threshold al
        - [x] 95%
        - [x] 97.5%
        - [x] 99%
        - [x] verifica della relazione $\tau_Q = \frac{Q}{1 - Q}$ dove $Q$ è il quantile scelto (0.95, 0.975, 0.99), $\tau_Q$ l'intervallo di ricorrenza medio, e confronto con l'evidenza dei dati
    - [x] peak-over-threshold definito come $pot = \mu \pm m \cdot \sigma$: ricavare analiticamente m come $m = \frac{q_x - \mu}{\sigma}$ dove $q_x$ è il quantile di ordine $x$

In [ ]:
import os
import time
import datetime
from typing import List
import itertools
import pickle
import math
import copy

import numpy as np
import pandas as pd
import pandas.testing as pt
import scipy.stats
import scipy.special as sfun
from scipy.stats import genextreme as gev
import sklearn.metrics as sm

from statsmodels.tsa import stattools
from statsmodels.graphics import tsaplots

import matplotlib.pyplot as pl
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns

from tqdm import tqdm
import ipdb

import numba

# import dello stimatore di Hill e del @timeit
from my_timeit import timeit

In [ ]:
%pdb on

In [ ]:
%load_ext line_profiler
%load_ext autoreload

In [ ]:
%autoreload 5

In [ ]:
return_type = ['pos', 'neg', 'abs']
quantile_type = ['95', '97.5', '99', 'evt']
distribution_type = ['weibull', 's-exp', 'q-exp']
stock_type = ['min_vol', 'max_vol']

colors = {
    'pos': 'seagreen',
    'neg': 'darkred',
    'abs': 'royalblue',
}

stock_colors = {
    'min_vol': 'palegoldenrod',
    'max_vol': 'coral',
}

legend_labels = {
    'pos': r'$r$',
    'neg': r'$-r$',
    'abs': r'$|r|$',
}

dist_colors = {
    'weibull': 'orchid',
    's-exp': 'orangered',
    'q-exp': 'mediumblue'
}

dist_labels = {
    'weibull': r'Weibull',
    's-exp': r's-exp',
    'q-exp': r'q-exp',
}

## 1. Importazione dati

### 1.1 Caricamento dati e unione in un singolo DataFrame

In [ ]:
data_path = "/Users/pietro/Google Drive/OptiRisk Thesis/data"
prices_path = os.path.join(data_path, 'prices', 'adjusted_prices_volume.csv')
index_path = os.path.join(data_path, 'prices', 'index.csv')

Conversione delle date e settaggio dell'index del dataframe

In [ ]:
prices = pd.read_csv(prices_path)
prices.loc[:, 'date'] = pd.to_datetime(prices['date'], format="%Y%m%d")
prices.index = prices['date']
prices.drop(columns=['date'], inplace=True)
prices.head()

Trasformiamola un una serie temporale, ogni riga una data, ogni colonna un'azione

In [ ]:
prices_ts = prices.pivot(columns='ravenpackId', values=['close'])
prices_ts.columns = prices_ts.columns.levels[1]
prices_ts.head()

Ora carico l'indice S&P500

In [ ]:
index = pd.read_csv(index_path)
index.loc[:, 'date'] = pd.to_datetime(index['date'], format="%Y%m%d")
index.index = index['date']
index.drop(columns=['date'], inplace=True)
index.head()

Ora carichiamo la corrispondenza codice - azienda:

In [ ]:
company_code_path = os.path.join(data_path, 'ravenpackId_name_ticker_sector.csv')
company_codes = pd.read_csv(company_code_path)

Ora dichiariamo le date in cui bisogna dividere i dati, visto che ci sono state crisi nei mesi/anni successivi:

In [ ]:
split_dates = {
    'insurance': datetime.datetime(1987, 1, 1), # insurance companies crisis
    'dot-com': datetime.datetime(2000, 1, 1), # dot-com bubble explodes
    'subprime-crisis': datetime.datetime(2007, 1, 1), # subprime crisis
    'subprime-crisis-start': datetime.datetime(2007, 1, 1), # subprime crisis
    'subprime-crisis-halfway': datetime.datetime(2008, 9, 1),
    'subprime-crisis-end': datetime.datetime(2010, 1, 1),
    'eu-debt': datetime.datetime(2011, 1, 1), # EU sovereign debt crisis
    'eu-debt-halfway': datetime.datetime(2012, 1, 1), # EU sovereign debt crisis
    'last_train': datetime.datetime(2017, 1, 1), 
}

Creo una funzione per dividere il dataset prima e dopo gli eventi critici:

In [ ]:
def divide(data: pd.DataFrame, before_date: datetime.datetime):
    """Split the data before and after the before_date."""
    before = data[data.index < before_date]
    after = data[data.index >= before_date]
    
    return before, after

Ora, se splitto sulla crisi finanziaria del 2007-2008 c'è il problema dei percentili, quindi utilizzo questa strategia:

- per il training set seleziono tutti i giorni compresi tra il 2005-01-03 (inizio dataset) e il 2008-09-01, cioè circa a metà della crisi, e dal 2012-01-01 al 2017-01-01
- il restante per il test set, cioè dal 2008-09-01 al 2012-01-01 e dal 2017-01-01 in avanti

### 1.2 Calcolo log-returns

In [ ]:
prices_ts_no_nan = prices_ts.dropna(axis='columns', how='any', inplace=False)
log_returns = np.log(prices_ts_no_nan).diff(periods=1).iloc[1:, :]
print(log_returns.shape)
log_returns.head()

In [ ]:
lr_train_1 = log_returns[log_returns.index < split_dates['subprime-crisis-halfway']]
lr_train_2 = log_returns[(log_returns.index >= split_dates['eu-debt-halfway']) & (log_returns.index < split_dates['last_train'])]

lr_train = pd.concat([lr_train_1, lr_train_2], axis='index')

lr_test_1 = log_returns[(log_returns.index >= split_dates['subprime-crisis-halfway']) & \
                        (log_returns.index < split_dates['eu-debt-halfway'])]
lr_test_2 = log_returns[log_returns.index >= split_dates['last_train']]

lr_test = pd.concat([lr_test_1, lr_test_2], axis='index')

## 2. Calcolo volatilità

Calcoliamo la volatilità in due modi:

- [x] con la standard deviation dei log-returns
- [x] con il beta coefficient rispetto all'indice S&P500

### 2.1 Volatilità con std. dev dei log-returns

In [ ]:
volatility = lr_train.std()
volatility.head()

In [ ]:
idx_max_vol = volatility.idxmax()
idx_min_vol = volatility.idxmin()

Ora capiamo a che stock appartengono

In [ ]:
company_max_vol = company_codes[company_codes.ravenpackId == idx_max_vol]
company_min_vol = company_codes[company_codes.ravenpackId == idx_min_vol]

print(f"Company with max volatility is \n{company_max_vol}\n")
print(f"Company with min volatility is \n{company_min_vol}")

In [ ]:
fig, ax = pl.subplots(nrows=2, ncols=2, figsize=(18, 10))

# max volatility
ax[0, 0].plot(prices_ts_no_nan.loc[lr_train_1.index, idx_max_vol], color='coral', label='train')
ax[0, 0].plot(prices_ts_no_nan.loc[lr_train_2.index, idx_max_vol], color='coral', label='__None__')
ax[0, 0].plot(prices_ts_no_nan.loc[lr_test_1.index, idx_max_vol], color='burlywood', label='test')
ax[0, 0].plot(prices_ts_no_nan.loc[lr_test_2.index, idx_max_vol], color='burlywood', label='__None__')
ax[0, 0].set_title(f'Most volatile stock | {company_max_vol.iloc[0, 1]}')

ax[1, 0].plot(log_returns.loc[lr_train_1.index, idx_max_vol], color='coral', label='train')
ax[1, 0].plot(log_returns.loc[lr_train_2.index, idx_max_vol], color='coral', label='__None__')
ax[1, 0].plot(log_returns.loc[lr_test_1.index, idx_max_vol], color='burlywood', label='test')
ax[1, 0].plot(log_returns.loc[lr_test_2.index, idx_max_vol], color='burlywood', label='__None__')

# min volatility
ax[0, 1].plot(prices_ts_no_nan.loc[lr_train_1.index, idx_min_vol], color='gold', label='train')
ax[0, 1].plot(prices_ts_no_nan.loc[lr_train_2.index, idx_min_vol], color='gold', label='__None__')
ax[0, 1].plot(prices_ts_no_nan.loc[lr_test_1.index, idx_min_vol], color='darkkhaki', label='test')
ax[0, 1].plot(prices_ts_no_nan.loc[lr_test_2.index, idx_min_vol], color='darkkhaki', label='__None__')
ax[0, 1].set_title(f'Least volatile stock | {company_min_vol.iloc[0, 1]}')

ax[1, 1].plot(log_returns.loc[lr_train_1.index, idx_min_vol], color='gold', label='train')
ax[1, 1].plot(log_returns.loc[lr_train_2.index, idx_min_vol], color='gold', label='__None__')
ax[1, 1].plot(log_returns.loc[lr_test_1.index, idx_min_vol], color='darkkhaki', label='test')
ax[1, 1].plot(log_returns.loc[lr_test_2.index, idx_min_vol], color='darkkhaki', label='__None__')

for i in range(ax.shape[0]):
    for j in range(ax.shape[1]):
        ax[i, j].legend()
        ax[i, j].set_xlabel('Year')
        if i == 0:
            ax[i, j].set_ylabel('Price in $')
        else:
            ax[i, j].set_ylabel('Log-return in $')

sns.despine()

### 2.3 Volatilità con beta coefficient

Il beta coefficient misura quanto un'asset è più o meno volatile rispetto a un benchmark di riferimento. In questo caso, è l'indice stesso S&P500, e si calcola così:

$$
\beta = \frac{Cov(r_a, r_b)}{Var(r_b)}
$$

dove $r_a$ sono i log-returns dell'asset, $r_b$ quelli del benchmark.

In [ ]:
index_log_returns = np.log(index.loc[:, ['close']]).diff(periods=1).iloc[1:, :]
index_log_returns.rename(columns={'close': 'index_SP500'}, inplace=True)
pt.assert_index_equal(index_log_returns.index, log_returns.index, check_names=False)

index_lr_train_1 = index_log_returns[index_log_returns.index < split_dates['subprime-crisis-halfway']]
index_lr_train_2 = index_log_returns[(index_log_returns.index >= split_dates['eu-debt-halfway']) & \
                                     (index_log_returns.index < split_dates['last_train'])]

index_lr_train = pd.concat([index_lr_train_1, index_lr_train_2], axis='index')

index_lr_test_1 = index_log_returns[(index_log_returns.index >= split_dates['subprime-crisis-halfway']) & \
                        (index_log_returns.index < split_dates['eu-debt-halfway'])]
index_lr_test_2 = index_log_returns[index_log_returns.index >= split_dates['last_train']]

index_lr_test = pd.concat([index_lr_test_1, index_lr_test_2], axis='index')

In [ ]:
train_all_together = pd.concat([lr_train, index_lr_train], axis='columns')
test_all_together = pd.concat([lr_test, index_lr_test], axis='columns')

In [ ]:
covariance = train_all_together.cov()
index_variance = train_all_together.loc[:, 'index_SP500'].var()

In [ ]:
beta = pd.DataFrame(
    data=[
        covariance.loc[company, 'index_SP500'] / index_variance
        for company in train_all_together.columns[:-1]
    ],
    index=train_all_together.columns[:-1],
    columns=['beta']
)

idx_max_beta = beta.idxmax()
idx_min_beta = beta.idxmin()

company_max_beta = company_codes[company_codes.ravenpackId == idx_max_beta.iloc[0]]
company_min_beta = company_codes[company_codes.ravenpackId == idx_min_beta.iloc[0]]

In [ ]:
print(f"Company with highest beta is \n{company_max_beta}\nwith beta {beta.loc[idx_max_beta, 'beta'][0]:5.3f}\n")
print(f"Company with lowest beta is \n{company_min_beta}\nwith beta {beta.loc[idx_min_beta, 'beta'][0]:5.3f}")

In [ ]:
fig, ax = pl.subplots(nrows=2, ncols=2, figsize=(18, 10))

# max beta
ax[0, 0].plot(prices_ts_no_nan.loc[lr_train_1.index, idx_max_beta], color='coral', label='train')
ax[0, 0].plot(prices_ts_no_nan.loc[lr_train_2.index, idx_max_beta], color='coral', label='__None__')
ax[0, 0].plot(prices_ts_no_nan.loc[lr_test_1.index, idx_max_beta], color='burlywood', label='test')
ax[0, 0].plot(prices_ts_no_nan.loc[lr_test_2.index, idx_max_beta], color='burlywood', label='__None__')
ax[0, 0].set_title(f'Most volatile stock | {company_max_beta.iloc[0, 1]}')

ax[1, 0].plot(log_returns.loc[lr_train_1.index, idx_max_beta], color='coral', label='train')
ax[1, 0].plot(log_returns.loc[lr_train_2.index, idx_max_beta], color='coral', label='__None__')
ax[1, 0].plot(log_returns.loc[lr_test_1.index, idx_max_beta], color='burlywood', label='test')
ax[1, 0].plot(log_returns.loc[lr_test_2.index, idx_max_beta], color='burlywood', label='__None__')

# min beta
ax[0, 1].plot(prices_ts_no_nan.loc[lr_train_1.index, idx_min_beta], color='gold', label='train')
ax[0, 1].plot(prices_ts_no_nan.loc[lr_train_2.index, idx_min_beta], color='gold', label='__None__')
ax[0, 1].plot(prices_ts_no_nan.loc[lr_test_1.index, idx_min_beta], color='darkkhaki', label='test')
ax[0, 1].plot(prices_ts_no_nan.loc[lr_test_2.index, idx_min_beta], color='darkkhaki', label='__None__')
ax[0, 1].set_title(f'Least volatile stock | {company_min_beta.iloc[0, 1]}')

ax[1, 1].plot(log_returns.loc[lr_train_1.index, idx_min_beta], color='gold', label='train')
ax[1, 1].plot(log_returns.loc[lr_train_2.index, idx_min_beta], color='gold', label='__None__')
ax[1, 1].plot(log_returns.loc[lr_test_1.index, idx_min_beta], color='darkkhaki', label='test')
ax[1, 1].plot(log_returns.loc[lr_test_2.index, idx_min_beta], color='darkkhaki', label='__None__')

for i in range(ax.shape[0]):
    for j in range(ax.shape[1]):
        ax[i, j].legend()
        ax[i, j].set_xlabel('Year')
        if i == 0:
            ax[i, j].set_ylabel('Price in $')
        else:
            ax[i, j].set_ylabel('Log-return in $')

sns.despine()

Ora, vediamo qual è la differenza tra l'azione più stabile identificata con la volatilità (Johnson & Johnson) e quella identificata con il $\beta$ (SLM Corp.)

In [ ]:
data = {
    'std': [
        volatility[idx_min_vol], # std J&J
        volatility[idx_min_beta[0]] # std SLM Corp
    ],
    'beta': [
        beta.loc[idx_min_vol, 'beta'], # beta J&J
        beta.loc[idx_min_beta[0], 'beta'] # beta SLM Corp
    ]
}

pd.DataFrame(
    data=data,
    index=['Johnson & Johnson', 'SLM Corp.']
)  # devo guardare i minimi eh!

Si vede che a livello di *std*, Johnson & Johnson è quasi la metà di SLM corp, mentre a livello di $\beta$ SLM Corp è il 50% in meno circa.

Dunque scegliamo come azioni:

- più volatile (beta):

In [ ]:
company_codes[company_codes.ravenpackId == idx_max_beta[0]]

- meno volatile (beta):

In [ ]:
company_codes[company_codes.ravenpackId == idx_min_beta[0]]

Creo quindi i dati di training e testing per queste due azioni

In [ ]:
split_key = 'eu-debt'

# divisione dataset in training (in-sample) e testing (out-of-sample), qui prima e dopo la crisi finanziaria 2007-2008
company_min_beta = company_codes[company_codes.ravenpackId == idx_min_beta['beta']]['ravenpackId'].iloc[0]
company_max_beta = company_codes[company_codes.ravenpackId == idx_max_beta['beta']]['ravenpackId'].iloc[0]

lr_train_max_vol = lr_train[company_max_beta]
lr_test_max_vol = lr_test[company_max_beta]

lr_train_min_vol = lr_train[company_min_beta]
lr_test_min_vol = lr_test[company_min_beta]

# returns contiene il training set
returns = {
    'max_vol': {
        'pos': lr_train_max_vol[lr_train_max_vol > 0.0],
        'neg': lr_train_max_vol[lr_train_max_vol < 0.0],
        'abs': lr_train_max_vol.abs()
    },
    'min_vol': {
        'pos': lr_train_min_vol[lr_train_min_vol > 0.0],
        'neg': lr_train_min_vol[lr_train_min_vol < 0.0],
        'abs': lr_train_min_vol.abs()
    }
}

returns_test = {
    'max_vol': {
        'pos': lr_test_max_vol[lr_test_max_vol > 0.0],
        'neg': lr_test_max_vol[lr_test_max_vol < 0.0],
        'abs': lr_test_max_vol.abs(),
    },
    'min_vol': {
        'pos': lr_test_min_vol[lr_test_min_vol > 0.0],
        'neg': lr_test_min_vol[lr_test_min_vol < 0.0],
        'abs': lr_test_min_vol.abs(),
    }
}

## 3. Stima dei parametri della distribuzione GEV

In questa sezione si replica la sezione 4.1 del paper.

Secondo la [Extreme Value Theory](https://en.wikipedia.org/wiki/Extreme_value_theory), trovare gli estremi significa trovare un gruppo di dati $x \geq x_t$ dove $x_t$ è l'*extreme value threshold*, e che soddisfi la GEV ([Generalized Extreme Values Distribution](https://en.wikipedia.org/wiki/Generalized_extreme_value_distribution)) che ha distribuzione cumulativa:

\begin{align*}
    G(x) &= exp\left[- \left(1 + \xi \frac{x - \mu}{\sigma}\right)^{-1/\xi}\right] \; for \; \xi \neq 0\\ 
    G(x) &= exp\left[-exp\left(\frac{x - \mu}{\sigma}\right)\right] \; for \; \xi = 0\\ 
\end{align*}

dove $\xi$ è lo *shape parameter* che determina la forma della coda, $1/\xi$ è il *tail exponent* della distribuzione.

Per trovare il threshold $x_t$, usiamo questo metodo:

1. sort dei dati (tutti i log-returns) in ordine discendente (o non-ascendente) per avere la sequenza $x_1 \geq x_2 \geq \ldots \geq x_n$
2. applicare lo [stimatore di Hill](https://en.wikipedia.org/wiki/Heavy-tailed_distribution#Hill's_tail-index_estimator) dove $n$ è il numero di samples, $k$ l'indice del k-esimo dato più grande (posizione k nella sequenza ordinata) chiamato *k-th order statistic*

$$\frac{1}{\hat{\xi}_{k,n}} = \frac{1}{\hat{\gamma}} = \frac{1}{k}\sum_{i=1}^{k}log\left(\frac{x_i}{x_{k+1}}\right)$$

3. calcolare la statistica di [Kolmogorov-Smirnov](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test#Kolmogorov%E2%80%93Smirnov_statistic) per quantificare il fitting tra la distribuzione GEV così ricavata (con $\hat{\xi}_{k,n}$ come esponente) e quella empirica della coda, dove la coda è rappresentata dai returns che sono $x \geq x_t$, cioé quelli ordinati discendenti con indice $i < k$
4. scegliere $k$ (e di conseguenza $x_t$ che non è altro che il k-esimo elemento dei return ordinati discendenti) come il valore associato alla *minima* statistica di Kolmogorov-Smirnov

Questo flusso viene applicato a tutti e 3 i tipi di returns considerati: positivi, negativi e assoluti.

Inoltre, usiamo una seconda versione di questo flusso, che è:

1. sort dei dati (tutti i log-returns) in ordine discendente (o non-ascendente) per avere la sequenza $x_1 \geq x_2 \geq \ldots \geq x_n$
2. scorrere nei returns ordinati con un indice $k$ che identifica il threshold scelto e fitting di una GEV sui *tail data*, dove per tail data si intendono tutti i returns con indice $i < k$
3. calcolare la statistica di [Kolmogorov-Smirnov](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test#Kolmogorov%E2%80%93Smirnov_statistic) per quantificare il fitting tra la distribuzione GEV e i dati
4. scegliere $k$ (e di conseguenza $x_t$ che non è altro che il k-esimo elemento dei return ordinati discendenti) come il valore associato alla *minima* statistica di Kolmogorov-Smirnov

Ciò che cambia tra i due flussi è che nel primo, calcoliamo *separatamente* $\gamma$ e gli altri parametri $\mu$ e $\sigma$, mentre nel secondo caso tutti insieme con una maximum-likelihood estimation.

Calcolo i return ordinati dal più grande al più piccolo per i positivi, negativi e assoluti per prima cosa e poi procedo.

In [ ]:
# 1. sorting dei returns - va fatto sia per il positivo, che per il negativo, che per gli assoluti
sorted_positive_lr_max_vol = returns['max_vol']['pos'].sort_values(ascending=False)
sorted_negative_lr_max_vol = (-returns['max_vol']['neg']).sort_values(ascending=False)
sorted_absolute_lr_max_vol = returns['max_vol']['abs'].sort_values(ascending=False)

sorted_positive_lr_min_vol = returns['min_vol']['pos'].sort_values(ascending=False)
sorted_negative_lr_min_vol = (-returns['min_vol']['neg']).sort_values(ascending=False)
sorted_absolute_lr_min_vol = returns['min_vol']['abs'].sort_values(ascending=False)

eps = 5e-6

sorted_lr = {
    'max_vol': {
        'pos': sorted_positive_lr_max_vol[sorted_positive_lr_max_vol >= eps],
        'neg': sorted_negative_lr_max_vol[sorted_negative_lr_max_vol >= eps],
        'abs': sorted_absolute_lr_max_vol[sorted_absolute_lr_max_vol >= eps],
    },
    'min_vol': {
        'pos': sorted_positive_lr_min_vol[sorted_positive_lr_min_vol >= eps],
        'neg': sorted_negative_lr_min_vol[sorted_negative_lr_min_vol >= eps],
        'abs': sorted_absolute_lr_min_vol[sorted_absolute_lr_min_vol >= eps],
    }
}

### 3.1 Stima di $\gamma$ separata

Cominciamo stimando $\gamma$.

Posso creare la mia funzione per stimare $\gamma$ esattamente come nel paper, cioè:

$$
\gamma = \frac{1}{k} \sum_{i = 1}{k} \left[ \ln x_{n + 1 - k} - \ln x_k \right]
$$

che però richiede i returns ordinati **ascendenti**.

Invece, uso la formula nell'altro paper [Tail index estimation, concentration and adaptivity](http://arxiv.org/abs/1503.05077), cioé

$$
\hat{\gamma}(k) = \frac{1}{k} \sum_{i = 1}^{k} \ln \left( \frac{x_i}{x_{k + 1}} \right)
$$

che richiede i returns ordinati **discendenti** come già sono, quindi meglio (ed è anche più facile da capire ed interpretare).

#### 3.1.1 Calcolo di $\gamma$

In [ ]:
def my_hill_estimator(lr: np.ndarray):
    """Get the estimation of gamma."""
    assert np.all(np.diff(lr) <= 0.0)  # ordinati discendenti, lr[i] - lr[i - 1] <= 0
    
    k_max = lr.shape[0]
    kappas = np.arange(1, k_max)
    
    gammas = np.zeros((kappas.shape[0], ), dtype=np.float64)
    
    for index, k in enumerate(kappas):  # index = k - 1
        ssum = np.sum(np.log(lr[:k - 1] / lr[k - 1 + 1]))        
        gammas[index] = (1 / k) * ssum
        
    return {
        'xis': gammas,
        'kappas': kappas
    }

Ora, calcoliamo $\xi = \gamma = -c$ con la mia funzione

In [ ]:
hill_estimation = {
    s_type: {
        ret_type: my_hill_estimator(sorted_lr[s_type][ret_type].values)
        for ret_type in return_type
    }
    for s_type in stock_type
}

#### 3.1.2 Plot di $\gamma$ al variare di k e dei returns

Plot di $\gamma$ al variare di $k$, cioè del threshold, e plot di $\gamma$ al variare dei return, per vedere come cambia a seconda di quale return si prenda come threshold:

In [ ]:
# plot delle stime di xi e gamma al variare di k
truncation = 100

# sulle righe in funzione di k o dei returns
# sulle colonne le due azioni, prima colonna la min_vol e seconda la max_vol
fig, ax = pl.subplots(nrows=2, ncols=2, figsize=(18, 14))

for i in range(2):  # sulle righe il tipo, gamma(k) o gamma(lr)
    
    for j, s_type in enumerate(stock_type):  # lavoriamo per colonne (azioni)
        for ret_type in return_type:  # tutti i tipi di return
            est = hill_estimation[s_type][ret_type]
            x = est['kappas'][truncation:] if i == 0 else sorted_lr[s_type][ret_type].values[1:][::-1][truncation:]
            y = est['xis'][truncation:]
            
            if i == 0:
                ax[i, j].plot(x, y, color=colors[ret_type], label=legend_labels[ret_type])
                ax[i, j].set(
                    title=r'{} - $\gamma$ as function of $k$'.format(s_type),
                    xlabel=r'$k$',
                    ylabel=r'$\gamma(k)$'
                )
            else:
                ax[i, j].semilogx(x, y, color=colors[ret_type], label=legend_labels[ret_type])
                ax[i, j].set(
                    title=r'{} - $\gamma$ as function of $r$'.format(s_type),
                    xlabel=r'$r$, $-r$, $|r|$',
                    ylabel=r'$\gamma(r)$'
                )
            
            ax[i, j].set_ylim([0, 5])
            ax[i, j].legend()

sns.despine()

#### 3.1.3 Fitting della GEV

Ora biogna passare a fittare la GEV e calcolare la statistica KS per ogni valore di $k$ (e quindi del return che funge da threshold, $x_t$).

Andremo poi a scegliere il valore di $k$ che minimizza la KS.

Creiamo allora una funzione apposita:

In [ ]:
@timeit
#@numba.jit(nopython=False, parallel=True, nogil=True)
def fit_gev(lr, xi, k, size=None, n_tries=1):
    """Find the best fitting GEV to the tail distribution of data in lr.
    lr MUST BE in descending order
    """
#     assert isinstance(xi, np.ndarray)
#     assert isinstance(k, np.ndarray)
#     assert xi.shape == k.shape
    
    # check descending
#     assert np.all(np.diff(lr) <= 0.0)
    n_xi = xi.shape[0]
    
    ks = np.zeros((n_xi, ))
    pvals = np.zeros((n_xi, ))
    fits = []
    
    print("Start fitting")
#     for i in numba.prange(n_xi):
    for i in range(n_xi):
        current_k = k[i]
        current_xi = xi[i]
        
        threshold = lr[current_k]
        tail_data = lr[:current_k]
        
        if size:
            ss = size
        else:
            ss = current_k
        
        fit = gev.fit(tail_data, fix_c=-current_xi)  # convenzioni diverse in SciPy
        fits.append(fit)
        
        k_stat_temp = np.zeros((n_tries, ))
        pval_temp = np.zeros((n_tries, ))
        
        c = fit[0]
        loc = fit[1]
        scale = fit[2]
        
        for j in range(n_tries):
            rvs = gev.rvs(c, loc, scale, ss)

            kk, pv = scipy.stats.ks_2samp(tail_data, rvs)
            k_stat_temp[j] = kk
            pval_temp[j] = pv
        
        ks[i] = np.mean(k_stat_temp)
        pvals[i] = np.mean(pval_temp)
        
    print("Fitting done")
    
    return {
        'ks': ks,
        'p': pvals,
        'fits': fits,
    }

ed applichiamola:

In [ ]:
load = False
npz_filename = f"./sp500/ks_stat-pvals_manual_split.npz"
pickle_filename = f"./sp500/fits_manual_split.pickle"

if load:  # just load the already computed KS and p-values
    loaded = np.load(npz_filename)
    
    with open(pickle_filename, 'rb') as infile:
        fits = pickle.load(infile)
    
    kolmog_smirn = {
        s_type: {
            ret_type: {
                'ks': loaded[s_type + '_ks_' + ret_type],
                'p': loaded[s_type + '_pvals_' + ret_type],
                'fits': fits[s_type][ret_type],
            }
            for ret_type in return_type
        }
        for s_type in stock_type
    }
    
    
else:  # compute them and save them
    size = 9999
    n_tries = 10
    print("\nFitting returns")

    kolmog_smirn = {
        s_type: {
            ret_type: fit_gev(
                sorted_lr[s_type][ret_type].values,
                hill_estimation[s_type][ret_type]['xis'],
                hill_estimation[s_type][ret_type]['kappas'],
                size=size,
                n_tries=n_tries,
            )
            for ret_type in return_type
        }
        for s_type in stock_type
    }
    
    to_save = dict()
    for s_type in stock_type:
        for ret_type in return_type:
            to_save[s_type + '_ks_' + ret_type] = kolmog_smirn[s_type][ret_type]['ks']
            to_save[s_type + '_pvals_' + ret_type] = kolmog_smirn[s_type][ret_type]['p']
    
    np.savez_compressed(npz_filename, **to_save)
    
    fits = {
        s_type: {
            ret_type: kolmog_smirn[s_type][ret_type]['fits']
            for ret_type in return_type
        }
        for s_type in stock_type
    }
    
    with open(pickle_filename, 'wb') as outfile:
        pickle.dump(fits, outfile)

In [ ]:
min_pval = 0.05

valid_pvals = {
    s_type: {
        ret_type: kolmog_smirn[s_type][ret_type]['p'] <= min_pval
        for ret_type in return_type
    }
    for s_type in stock_type
}

# trovo gli indici minimi
i_min_ks = dict()

for s_type in stock_type:
    i_min_ks[s_type] = dict()
    for ret_type in return_type:
        # indici booleani di validità del p-value
        mask = valid_pvals[s_type][ret_type]

        # copio la KS per quel return
        y = copy.deepcopy(kolmog_smirn[s_type][ret_type]['ks'])

        # dove il p-value > 0.05, setto la KS al massimo così non viene considerata
        y[np.logical_not(mask)] = np.max(y)

        # trovo gli indici di minima distanza KS per questo return
        i_min_ks[s_type][ret_type] = np.argmin(y)

Le figure seguenti mostrano l'andamento di $d_{KS}$ in funzione di k e dei returns ordinati, su scala $x$ semilogaritmica, per entrambi i modi di calcolare il fitting della GEV

In [ ]:
# plot della statistica KS al variare del sorted return e di k
truncation = 100
end = -170
p_labels = {
    'pos': r'valid $p$ for $r$',
    'neg': r'valid $p$ for $-r$',
    'abs': r'valid $p$ for $|r|$',
}

p_height = {
    'pos': 0.0,
    'neg': -0.01,
    'abs': -0.02
}

markersize = 0.7
p_markersize = 0.7

fig, ax = pl.subplots(nrows=2, ncols=len(stock_type), figsize=(22, 14))

for i in range(ax.shape[0]):  # sulle righe d_KS(r) e d_KS(k)
    for j, s_type in enumerate(stock_type):
        for ret_type in return_type:
            x = sorted_lr[s_type][ret_type].values[1:] if i == 0 else hill_estimation[s_type][ret_type]['kappas']
            y = kolmog_smirn[s_type][ret_type]['ks']

            mask = valid_pvals[s_type][ret_type]

            x_ok = x[mask]
            y_ok = y[mask]

            if i == 0:
                ax[i, j].semilogx(
                    x,
                    y,
                    color=colors[ret_type],
                    linestyle='',
                    marker='.',
                    markersize=markersize,
                    label=legend_labels[ret_type]
                )

                ax[i, j].semilogx(
                    x_ok,
                    p_height[ret_type] * np.ones((len(x_ok, ))),
                    color=colors[ret_type],
                    linestyle='',
                    marker='.',
                    markersize=p_markersize
                )

                ax[i, j].axvline(
                    x[i_min_ks[s_type][ret_type]],
                    linestyle='-.',
                    color=colors[ret_type],
                    alpha=0.7,
                    label=r'min $d_{ks}$ for ' + legend_labels[ret_type]
                )
            else:
                ax[i, j].plot(
                    x,
                    y,
                    color=colors[ret_type],
                    linestyle='',
                    marker='.',
                    markersize=markersize,
                    label=legend_labels[ret_type]
                )

                ax[i, j].plot(
                    x_ok,
                    p_height[ret_type] * np.ones((len(x_ok, ))),
                    color=colors[ret_type],
                    linestyle='',
                    marker='.',
                    markersize=p_markersize
                )

                ax[i, j].axvline(
                    x[i_min_ks[s_type][ret_type]],
                    linestyle='-.',
                    color=colors[ret_type],
                    alpha=0.7,
                    label=r'min $d_{ks}$ for ' + legend_labels[ret_type]
                )
            
        ax[i, j].legend()

ax[0, 0].set_ylim([-0.03, 0.5])
ax[0, 1].set_ylim([-0.03, 0.5])

ax[0, 0].set_xlim([1e-4, 1e-1])
ax[0, 1].set_xlim([1e-4, 1e-1])

ax[1, 0].set_ylim([-0.03, 0.5])
ax[1, 1].set_ylim([-0.03, 0.5]);

sns.despine()

Le linee sottostanti le figure, composte in realtà da punti, identificano quei valori per cui il test di Kolmogorov-Smirnov ha dato un _p_-value $p \leq 0.05$, ed è quindi ritenuto statisticamente valido. Si nota come agli estremi di $k$ e dei returns il *p*-value non sia significativo e ci siano grosse oscillazioni, probabilmente dovute a instabilità numeriche nel calcolo della maximum likelihood.

### 3.2 Minima $d_{KS}$ per trovare il miglior fit della GEV

Ottimo, ora bisogna selezionare il minimo valore della statistica KS $d_{KS}$ che abbia un p-value valido ($p < 0.05$).

In [ ]:
def find_min_dks(d_ks, pvals, min_pval=min_pval):
    dks = d_ks.copy()
    invalid = pvals > min_pval
    
    dks[invalid] = np.min(dks) + 1
    i_min = np.argmin(dks)
    
    return dks[i_min], i_min

In [ ]:
min_kolmog_smirn = {
    s_type: {
        ret_type: find_min_dks(kolmog_smirn[s_type][ret_type]['ks'], kolmog_smirn[s_type][ret_type]['p'])
        for ret_type in return_type
    }
    for s_type in stock_type
}

threshold_evt = {
    s_type: {
        'pos': sorted_lr[s_type]['pos'][min_kolmog_smirn[s_type]['pos'][1]],
        'neg': sorted_lr[s_type]['neg'][min_kolmog_smirn[s_type]['neg'][1]],
        'abs': sorted_lr[s_type]['abs'][min_kolmog_smirn[s_type]['abs'][1]],
    }
    for s_type in stock_type
}

title_format = "{:>15}"*5
row_format = "{:>15}{:>15}{:>15.4f}{:>15}{:>15.6f}"

print(title_format.format('Stock type', 'Return type', 'Min d_KS', 'i', 'Return'))
print("-"*15 * 5)
for s_type in stock_type:
    print("-"*15 * 5)
    for ret_type in return_type:
        print(row_format.format(
            s_type,
            ret_type,
            min_kolmog_smirn[s_type][ret_type][0],
            min_kolmog_smirn[s_type][ret_type][1],
            threshold_evt[s_type][ret_type]
        ))

Ora vediamo le GEV fittate sui 3 returns, per le due azioni

In [ ]:
# preparo i dati per plottare
titles = {
    'pos': 'Positive extreme returns',
    'neg': 'Negative extreme returns',
    'abs': 'Absolute extreme returns',
}

xlabels = {
    'pos': r'Positive extreme returns $r$',
    'neg': r'Negative extreme returns $-r$',
    'abs': r'Absolute extreme returns  $|r|$',
}

labels = {
    'pos': r'$r$',
    'neg': r'$-r$',
    'abs': r'$|r|$',
}

# plot
fig, ax = pl.subplots(nrows=len(stock_type), ncols=len(return_type), figsize=(18, 10), sharex=False, sharey=True)

# sulle righe i due flussi
for i, s_type in enumerate(stock_type):
    
    # sulle colonne i 3 tipi di returns e le loro GEV fittate
    for j, ret_type in enumerate(return_type):
        i_min = min_kolmog_smirn[s_type][ret_type][1]

        data = sorted_lr[s_type][ret_type].values[:i_min]
        best_fit = fits[s_type][ret_type][i_min]
        
        sns.distplot(
            data,
            color=colors[ret_type],
            label=legend_labels[ret_type],
            kde=False,
            norm_hist=True,
            ax=ax[i, j]
        )
        
        title = s_type
        _, b = ax[i, j].xaxis.get_data_interval()
        x = np.linspace(0, b, 1000)
        pdf = gev.pdf(x, *best_fit)
        ax[i, j].plot(x, pdf, color=colors[ret_type], label='GEV pdf')
        
        ax[i, j].set_title(title)
        ax[i, j].set_xlabel(xlabels[ret_type])
        ax[i, j].legend()

sns.despine()

Finora abbiamo quindi ottenuto:

- le distribuzioni di probabilità degli extreme returns (positivi, negativi, assoluti)
- i threshold che massimizzano il fitting della distribuzione *GEV* sugli extreme returns. Tali threshold possono essere quindi usati per determinare quali movimenti siano estremi e quali no

Concludiamo quindi confrontando i threshold così ottenuti con i threshold del 95% percentile che abbiamo utilizzato finora per le azioni S&P500, e con quello che si otterrebbe ad utilizzare il $k^*$ calcolato con lo stimatore di Hill.

In [ ]:
# calcolo dei threshold
# quantili: attenzione che sono TUTTI POSITIVI
thresholds = {
    s_type: {
        ret_type: {
            q_type: returns[s_type][ret_type].abs().quantile(float(q_type) / 100)
            for q_type in quantile_type[:-1]
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

for s_type in stock_type:
    for ret_type in return_type:
        thresholds[s_type][ret_type]['evt'] = abs(threshold_evt[s_type][ret_type])

extremes = {
    s_type: {
        ret_type: {
            q_type: (returns[s_type][ret_type].abs() >= thresholds[s_type][ret_type][q_type]).astype(np.int8)
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

extremes_test = {
    s_type: {
        ret_type: {
            q_type: (returns_test[s_type][ret_type].abs() >= thresholds[s_type][ret_type][q_type]).astype(np.int8)
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

Vediamo come sono i threshold graficamente

In [ ]:
fig, ax = pl.subplots(nrows=2, ncols=1, figsize=(18, 10))

for i, s_type in enumerate(stock_type):
    if s_type == 'min_vol':
        y = log_returns[idx_min_beta['beta']]
    else:
        y = log_returns[idx_max_beta['beta']]
        
    x = y.index
        
    y_train = y[x < split_dates[split_key]]
    y_test = y[x >= split_dates[split_key]]
    
    ax[i].plot(
        y_train,
        label=s_type,
        color=stock_colors[s_type],
        alpha=0.75,
        linestyle='',
        marker='.'
    )
    
    ax[i].plot(
        y_test,
        label='__None__',
        color=stock_colors[s_type],
        alpha=0.75,
        linestyle='',
        marker='.'
    )
    
    ax[i].axvline(
        split_dates['subprime-crisis-halfway'],
        color='grey',
        linestyle='-',
    )
    
    ax[i].axvline(
        split_dates['eu-debt-halfway'],
        color='grey',
        linestyle='-',
    )
    
    ax[i].axvline(
        split_dates['last_train'],
        color='grey',
        linestyle='-',
    )
    
    # ora i percentili
    ax[i].plot(
        x, -thresholds[s_type]['neg']['95'] * np.ones(len(x)), color=colors['neg'], linestyle='--', label=r'95% percentile, $-r$'
    )
    ax[i].plot(
        x, thresholds[s_type]['pos']['95'] * np.ones(len(x)), color=colors['pos'], linestyle='--', label=r'95% percentile, $r$'
    )
    ax[i].plot(
        x, -thresholds[s_type]['neg']['evt'] * np.ones(len(x)), color=colors['neg'], linestyle=':', label=r'EVT, $-r$'
    )
    ax[i].plot(
        x, -thresholds[s_type]['pos']['evt'] * np.ones(len(x)), color=colors['pos'], linestyle=':', label=r'EVT, $r$'
    )
    
    ax[i].set_title(str.capitalize(s_type))
    ax[i].set_ylabel('Log Returns')
    ax[i].legend()
    
sns.despine()

Come si può vedere dal plot, i percentili sono molto più stringenti rispetto al valore che massimizza il fitting della *GEV*, mentre i threshold calcolati con il solo stimatore di Hill sono più stringenti dei percentili.

Vediamo anche di confermarlo con un po' di numeri:

In [ ]:
def get_percent(data, thresh_low, thresh_up):
    indexes = np.logical_or(data.values <= thresh_low, data.values >= thresh_up)
    num = len(data[indexes])
    denom = len(data)
    
    return num / denom

perc = dict()
extremes_with_percentiles = dict()
extremes_with_evt = dict()

for s_type in stock_type:
    idx = idx_min_beta['beta'] if s_type == 'min_vol' else idx_max_beta['beta']
    perc[s_type] = log_returns[idx].quantile(q=[0.05, 0.95])
    
    extremes_with_percentiles[s_type] = get_percent(log_returns[idx], perc[s_type][0.05], perc[s_type][0.95])
    extremes_with_evt[s_type] = get_percent(log_returns[idx], -thresholds[s_type]['neg']['evt'], thresholds[s_type]['pos']['evt'])


print("{:>20}{:>20}{:>15}".format('Stock Type', 'Threshold type', 'Extremes %'))
print("-"*55)
for s_type in stock_type:
    print("-" * 55)
    print("{:>20}{:>20}{:>15.3f}".format(s_type, 'percentile 5-95 %', extremes_with_percentiles[s_type]))
    print("{:>20}{:>20}{:>15.3f}".format(s_type, 'EVT', extremes_with_evt[s_type]))

In pratica, vuol dire che se usassimo i valori di threshold ricavati dalla *EVT* avremmo un dataset sicuramente più bilanciato, ma c'è da chiedersi se si possano effettivamente allora considerare "estremi". Non è troppo "inclusivo" un tale threshold?

**I threshold della EVT non vanno bene!**

### 3.5 Calcolo dei $\tau_Q$ e delle $Q$

Calcoliamoli per poi usarli nella maximum likelihood estimation.

In [ ]:
# creo i tau e calcolo i Q
tau_q_95 = 1.0 / (1.0 - 0.95)
tau_q_975 = 1.0 / (1.0 - 0.975)
tau_q_99 = 1.0 / (1.0 - 0.99)

# calcolo i quantili equivalenti ai threshold della EVT
Q = {
    s_type: {
        ret_type: 1.0 - (sum(returns['min_vol'][ret_type].abs() >= abs(threshold_evt[s_type][ret_type])) / len(returns[s_type][ret_type]))
        for ret_type in return_type
    }
    for s_type in stock_type
}

tau_q = {
    s_type: {
        ret_type: {
            '95': tau_q_95,
            '97.5': tau_q_975,
            '99': tau_q_99,
            'evt': 1.0 / (1.0 - Q[s_type][ret_type])
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

## 4. Calcolo degli intervalli di ricorrenza e plot della loro distribuzione

Ora che abbiamo i threshold possiamo calcolare gli intervalli di ricorrenza e vederne la distribuzione.

In [ ]:
def get_recurrence_intervals(is_extreme: pd.DataFrame):
    """Get the recurrence intervals durations between extremes.
    
    Parameters
    ----------
    is_extreme: pd.DataFrame
        a DataFrame with the date on the index and 1 if the return at time t is extreme,
        0 otherwise. Must contain a single column named 'extreme'
    """
    assert isinstance(is_extreme.index, pd.DatetimeIndex)
    assert len(is_extreme.columns) == 1
    
    # convert to int
    data = is_extreme.astype(np.int8)
    data.loc[:, 'date'] = data.index
    data.index = pd.RangeIndex(len(is_extreme))
    
    data_is_extreme = data[data[data.columns[0]] == 1]
    
    intervals = []
    for i in range(1, len(data_is_extreme)):
        last_time = data_is_extreme.date.iloc[i - 1]
        current_time = data_is_extreme.date.iloc[i]
        
        n_days = data_is_extreme.index[i] - data_is_extreme.index[i - 1]
        
        intervals.append((last_time, current_time, n_days))
        
    return intervals

I quantili vanno presi al 95%, 97.5%, 99%. Creo quindi il `dict` che contiene gli intervalli di ricorrenza, organizzati secondo il tipo di return ed il tipo di threshold (quantile o evt):

In [ ]:
# calcolo intervalli di ricorrenza - training set
tmp_rec_int = {
    s_type: {
        ret_type: {
            q_type: get_recurrence_intervals(pd.DataFrame(extremes[s_type][ret_type][q_type]))
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

recurrence_intervals = {
    s_type: {
        ret_type: {
            q_type: pd.DataFrame(data={
                'last_extreme': [x[0] for x in tmp_rec_int[s_type][ret_type][q_type]],
                'current_extreme': [x[1] for x in tmp_rec_int[s_type][ret_type][q_type]],
                'n_days': [x[2] for x in tmp_rec_int[s_type][ret_type][q_type]],
            })
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

# calcolo intervalli di ricorrenza - testing set
tmp_rec_int_test = {
    s_type: {
        ret_type: {
            q_type: get_recurrence_intervals(pd.DataFrame(extremes_test[s_type][ret_type][q_type]))
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

recurrence_intervals_test = {
    s_type: {
        ret_type: {
            q_type: pd.DataFrame(data={
                'last_extreme': [x[0] for x in tmp_rec_int_test[s_type][ret_type][q_type]],
                'current_extreme': [x[1] for x in tmp_rec_int_test[s_type][ret_type][q_type]],
                'n_days': [x[2] for x in tmp_rec_int_test[s_type][ret_type][q_type]],
            })
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

### 4.1 Plot istogrammi intervalli di ricorrenza

Ora visualizzo graficamente la lunghezza degli intervalli di ricorrenza con degli istogrammi, rispettivamente per i returns positivi, negativi ed assoluti.

In [ ]:
hist_labels = {
    '95': '95%',
    '97.5': '97.5%',
    '99': '99%',
    'evt': 'EVT',
}

titles = {
    'pos': r'Positive $r$',
    'neg': r'Negative $-r$',
    'abs': r'Absolute $|r|$',
}

y_lims = {
    'pos': [0.0, 0.15],
    'neg': [0.0, 0.15],
    'abs': [0.0, 0.08],
}

fig, ax = pl.subplots(nrows=2, ncols=3, figsize=(18, 12))

# riga 1: training set
for i, ret_type in enumerate(return_type):
    for q_type in quantile_type:
        curr = recurrence_intervals['min_vol'][ret_type][q_type].n_days
        sns.distplot(curr, kde=False, norm_hist=True, label=hist_labels[q_type], ax=ax[0, i])

    ax[0, i].legend()
    ax[0, i].set(title='min_vol | ' + titles[ret_type] + " | training set", ylim=y_lims[ret_type])

# riga 2: testing set
for i, ret_type in enumerate(return_type):
    for q_type in quantile_type:
        curr = recurrence_intervals_test['min_vol'][ret_type][q_type].n_days
        sns.distplot(curr, kde=False, norm_hist=True, label=hist_labels[q_type], ax=ax[1, i])

    ax[1, i].legend()
    ax[1, i].set(title='min_vol | ' + titles[ret_type] + " | testing set", ylim=y_lims[ret_type])

sns.despine()

In [ ]:
# ora per la massima volatile
fig, ax = pl.subplots(nrows=2, ncols=3, figsize=(18, 12))

# riga 1: training set
for i, ret_type in enumerate(return_type):
    for q_type in quantile_type:
        curr = recurrence_intervals['max_vol'][ret_type][q_type].n_days
        sns.distplot(curr, kde=False, norm_hist=True, label=hist_labels[q_type], ax=ax[0, i])

    ax[0, i].legend()
    ax[0, i].set(title='max_vol | ' + titles[ret_type] + " | training set", ylim=y_lims[ret_type])

# riga 2: testing set
for i, ret_type in enumerate(return_type):
    for q_type in quantile_type:
        curr = recurrence_intervals_test['max_vol'][ret_type][q_type].n_days
        sns.distplot(curr, kde=False, norm_hist=True, label=hist_labels[q_type], ax=ax[1, i])

    ax[1, i].legend()
    ax[1, i].set(title='max_vol | ' + titles[ret_type] + " | testing set", ylim=y_lims[ret_type])

sns.despine()

### 4.2 Creazione delle tabelle come nel paper

Ora creo le tabelle riassuntive come a pagina 9 del paper di [Jiang et al](https://doi.org/10.1080/14697688.2017.1373843).

Prima mi creo due funzioncine e poi le chiamo.

In [ ]:
def get_single_table(intervals: pd.DataFrame, returns: pd.DataFrame, ret_type: str, thresh: float, col_name='perc'):
    """Get a single panel sub-table."""
    obsv = int(intervals.shape[0])
    mean = intervals['n_days'].mean()
    median = intervals['n_days'].median()
    std_dev = intervals['n_days'].std()
    skewness = intervals['n_days'].skew()
    kurtosis = intervals['n_days'].kurt()
    
    ret_mean = returns.mean()
    ret_std_dev = returns.std()
    
    m = (thresh - ret_mean) / ret_std_dev
#     print(f"\nRet_type: {ret_type}, q_type: {col_name}")
#     print(f"Threshold: {thresh:5.4f}, Mean: {ret_mean:5.4f}, m: {m:5.4f}")
        
    acf, qstat, pvals = stattools.acf(intervals['n_days'].values, qstat=True, nlags=30)
    rho1 = acf[1]
    _, p_rho1 = scipy.stats.pearsonr(
        intervals['n_days'].values[1:],
        intervals['n_days'].shift(periods=1).values[1:],
    )
    
    rho5 = acf[5]
    _, p_rho5 = scipy.stats.pearsonr(
        intervals['n_days'].values[5:],
        intervals['n_days'].shift(periods=5).values[5:],
    )

    Q30 = qstat[-1]
    p_Q30 = pvals[-1]
    
    index = pd.Index(data=[
        'm',
        'obsv',
        'mean',
        'median',
        'stdev',
        'skew',
        'kurt',
        'rho(1)',
        'p-value(rho1)',
        'rho(5)',
        'p-value(rho5)',
        'Q(30)',
        'p-value(Q30)',
    ])
    
    result = pd.DataFrame(data=[
        [m],
        [obsv],
        [mean],
        [median],
        [std_dev],
        [skewness],
        [kurtosis],
        [rho1],
        [p_rho1],
        [rho5],
        [p_rho5],
        [Q30],
        [p_Q30],
    ],
    index=index,
    columns=[col_name])
    
    return result

Il titolo è la sezione della tabella (Negative/Positive/Absolute), i quantili sono quelli che mi interessano e finiranno sulle colonne della tabella ed il risultato è un `dict` che ha come chiavi i titoli.

In [ ]:
tables = {
    s_type: {
        ret_type: {
            q_type: get_single_table(recurrence_intervals[s_type][ret_type][q_type],
                                     returns[s_type][ret_type],
                                     ret_type,
                                     thresh=thresholds[s_type][ret_type][q_type],
                                     col_name=q_type)
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

panels = {
    s_type: {
        ret_type: pd.concat([tables[s_type][ret_type][q_type] for q_type in quantile_type], axis='columns')
        for ret_type in return_type
    }
    for s_type in stock_type
}

Visualizziamo le tabelle:

In [ ]:
panels['min_vol']['pos']

In [ ]:
panels['min_vol']['neg']

In [ ]:
panels['min_vol']['abs']

In [ ]:
panels['max_vol']['pos']

In [ ]:
panels['max_vol']['neg']

In [ ]:
panels['max_vol']['abs']

### 4.3 Plot degli autocorrelogrammi

Vediamo con gli [autocorrelogammi](https://en.wikipedia.org/wiki/Correlogram) se c'è autocorrelazione nelle serie dei *recurrence interval*.

Prima quelli dell'azione meno volatile:

In [ ]:
# sulle righe il tipo di threshold, sulle colonne il tipo di return
s_type = 'min_vol'

fig, ax = pl.subplots(nrows=2, ncols=3, figsize=(27, 10))
fig.suptitle(f"{s_type} - Autocorrelation plots", fontsize=16)

tsaplots.plot_acf(recurrence_intervals[s_type]['neg']['95']['n_days'].values, lags=30, ax=ax[0][0], title=r'$-r$, 95%')
tsaplots.plot_acf(recurrence_intervals[s_type]['pos']['95']['n_days'].values, lags=30, ax=ax[0][1], title=r'$r$, 95%')
tsaplots.plot_acf(recurrence_intervals[s_type]['abs']['95']['n_days'].values, lags=30, ax=ax[0][2], title=r'$|r|$, 95%')
ax[0, 0].set_ylabel("Pearson $R$")

tsaplots.plot_acf(recurrence_intervals[s_type]['neg']['evt']['n_days'].values, lags=30, ax=ax[1][0], title=r'$-r$, EVT')
tsaplots.plot_acf(recurrence_intervals[s_type]['pos']['evt']['n_days'].values, lags=30, ax=ax[1][1], title=r'$r$, EVT')
tsaplots.plot_acf(recurrence_intervals[s_type]['abs']['evt']['n_days'].values, lags=30, ax=ax[1][2], title=r'$|r|$, EVT')
ax[1, 0].set_ylabel("Pearson $R$")

for a in ax[1]:
    a.set_xlabel('lag in the recurrence interval array')

sns.despine()

In [ ]:
# sulle righe il tipo di threshold, sulle colonne il tipo di return
s_type = 'max_vol'

fig, ax = pl.subplots(nrows=2, ncols=3, figsize=(27, 10))
fig.suptitle(f"{s_type} - Autocorrelation plots", fontsize=16)

tsaplots.plot_acf(recurrence_intervals[s_type]['neg']['95']['n_days'].values, lags=30, ax=ax[0][0], title=r'$-r$, 95%')
tsaplots.plot_acf(recurrence_intervals[s_type]['pos']['95']['n_days'].values, lags=30, ax=ax[0][1], title=r'$r$, 95%')
tsaplots.plot_acf(recurrence_intervals[s_type]['abs']['95']['n_days'].values, lags=30, ax=ax[0][2], title=r'$|r|$, 95%')
ax[0, 0].set_ylabel("Pearson $R$")

tsaplots.plot_acf(recurrence_intervals[s_type]['neg']['evt']['n_days'].values, lags=30, ax=ax[1][0], title=r'$-r$, EVT')
tsaplots.plot_acf(recurrence_intervals[s_type]['pos']['evt']['n_days'].values, lags=30, ax=ax[1][1], title=r'$r$, EVT')
tsaplots.plot_acf(recurrence_intervals[s_type]['abs']['evt']['n_days'].values, lags=30, ax=ax[1][2], title=r'$|r|$, EVT')
ax[1, 0].set_ylabel("Pearson $R$")

for a in ax[1]:
    a.set_xlabel('lag in the recurrence interval array')

sns.despine()

Per interpretare i plot, bisogna ricordare che:

- sulle $x$ c'è il lag della serie temporale relativa ai giorni tra i movimenti estremi, cioè quella degli intervalli di ricorrenza. Vuol dire che $x=22$ significa il 22° intervallo di ricorrenza visto nel passato, prima di quello attuale, non 22 giorni prima di oggi. La distanza in giorni potrebbe anche essere un anno o più.
- sulle y c'è la correlazione di Pearson $R$

Apparentemente **non c'è autocorrelazione nei recurrence intervals**, con nessun quantile, a parte sporadici (es il 95%)

### 4.4 Verifica relazione empirica

Verifichiamo ora la relazione empirica $\tau_Q = \frac{Q}{1 - Q}$ dove $Q$ è il quantile scelto (0.95, 0.975, 0.99), $\tau_Q$ l'intervallo di ricorrenza medio

In [ ]:
title_format = "{:>15}"*6
row_format = "{:>15}{:>15.3f}{:>15}{:>15.3f}{:>15.3f}" + "{:>14.3f}%"
print(title_format.format('Stock type', 'Quantile', 'Return type', 'tau_q', 'True mean', 'Error %'))
print('-' * 15 * 6)

for s_type in stock_type:
    print('-' * 15 * 6)
    for q_type in quantile_type[:-1]:
        for i, name in enumerate(return_type):        
            data_mean = recurrence_intervals[s_type][name][q_type]['n_days'].mean()

            q = float(q_type) / 100.0
            tau = 1.0 / (1.0 - q)

            perc_diff = (tau - data_mean) / data_mean

            print(row_format.format(s_type, q, name, tau, data_mean, perc_diff * 100))

            if i == len(return_type) - 1:
                print("")

**La relazione non vale in questo caso.**

# 5. Determinazione della *Hazard Probability*

Gli autori definiscono la *hazard probability* come

\begin{equation}
    W(\Delta t | t) = \frac{\int_t^{t + \Delta t} p(\tau)d\tau}{\int_t^{\infty}p(\tau)d\tau}
\end{equation}

dove $p(\tau)$ è la distribuzione di probabilità (`pdf` per scipy).

La hazard probability definisce la probabilità che, dato che si è verificato un evento estremo $t$ giorni nel passato, ci sia un tempo di attesa $\Delta t$ ulteriore prima di un altro evento estremo.
Se consideriamo $W(1 | t)$ è simile al problema che abbiamo affrontato con la rete neurale.

Ora, nota la ditribuzione $p(\tau)$, si può derivare analiticamente l'integrale. Il problema è quindi: come trovare $p(\tau)$, e che forma ha?

Gli autori utilizzano una [stretched exponential distribution](https://en.wikipedia.org/wiki/Stretched_exponential_function), una [*q*-exponential distribution](https://en.wikipedia.org/wiki/Q-exponential_distribution) ed una [Weibull distribution](https://it.wikipedia.org/wiki/Distribuzione_di_Weibull). I parametri delle 3 distribuzioni vengono stimati tramite MLE.

Il flusso è il seguente:

1. scegli una distribuzione (s-exp, q-exp, Weibull)
2. riformula la parametrizzazione in funzione solo  dello *shape parameter*
3. calcola la log-likelihood utilizzando una semplice ricerca a griglia sui parametri liberi
4. i parametri che forniscono la massima log-likelihood sono quelli cercati, e trova la formula teorica della *hazard probability* con le equazioni del paper

Cominciamo con la Weibull, ma prima creiamo una funzione che calcoli la *hazard probability* empirica, con la formula

$$
W_{emp}(\Delta t | t) = \frac{\#(t < \tau \leq t + \Delta t)}{\#(\tau > t)}
$$

dove al numeratore c'è il numero di recurrence intervals con valore compreso in $(t, t + \Delta t]$, al denominatore il numero di recurrence intervals con valore maggiore di $t$, cioè nel range $(t, +\infty)$.

In [ ]:
def get_empirical_hazard_prob(rec_ints: np.ndarray, t, delta_t):
    """Compute the empirical hazard probability."""
    assert isinstance(rec_ints, np.ndarray)
    num = np.sum(np.logical_and(rec_ints > t, rec_ints <= t + delta_t))
    denom = np.sum(rec_ints > t)
    
    return num / denom

## 5.1 Fitting della Weibull

In `scipy.stats` è definita come

\begin{eqnarray}
&f(x, c) = c x^{c - 1} e^{-x^{c}} \\
&f(x, c, loc, scale) = \frac{1}{scale}f\left(\frac{x - loc}{scale}, c\right)
\end{eqnarray}

dove $c$ è lo *shape parameter*. Nel paper invece è

\begin{equation}
f(x, \beta, \alpha) = \frac{\alpha}{\beta} \left( \frac{\tau}{\beta} \right)^{\alpha - 1} e^{-\left( \frac{\tau}{\beta} \right)^{\alpha}}
\end{equation}

quindi la corrispondenza è

\begin{eqnarray}
&loc = 0 \\
&\beta = scale \\
&shape = c = \alpha
\end{eqnarray}

Ora dobbiamo stimare i parametri della Weibull con una maximum log-likelihood estimation (*MLE*). Riscrivendoli in funzione di $\tau_Q$ e $\beta = scale$ abbiamo

\begin{eqnarray}
&\beta = \frac{\tau_Q}{\Gamma \left( 1 + \frac{1}{\alpha} \right)} \\
cioè \\
&\beta = scale = \frac{\tau_Q}{\Gamma \left( 1 + \frac{1}{c} \right)}
\end{eqnarray}

Ricordiamo che $\tau_Q = \frac{1}{1 - Q}$ dove $Q$ è il quantile.

A questo punto la MLE ha formula:

$$ ln(L_w) = n \cdot ln\left( \frac{c}{\beta} \right) + \sum_{i=1}^{n} \left[ (c - 1) ln\left( \frac{\tau_i}{\beta} \right) - \left( \frac{\tau_i}{\beta} \right)^c \right] $$

dove $n$ è il numero di recurrence intervals, $t_i$ il corrispondente valore dell'intervallo di ricorrenza (es: 14 giorni, 4 giorni...).

Il flow è quindi, in questo caso:

1. a seconda del percentile (95% o EVT) calcolare $Q$ e quindi $\tau_Q$
2. utilizzare una ricerca con step $1e-6$ sul parametro $c = \alpha$, il quale risulta in un certo valore di $\beta$
3. utilizzare quei valori di $c$ e di $\beta$ nella MLE
4. trovare il massimo della MLE ed i corrispondenti valori di $c$ e $\beta$
5. urrà! Ora possiamo usarli nella *pdf* della distribuzione Weibull per ottenere l'hazard $W(\Delta t | t)$

In [ ]:
@timeit
@numba.jit(nopython=True, parallel=True, nogil=True)
def mle_weibull(rec_ints: np.ndarray, c: np.ndarray, beta: np.ndarray):
    """MLE estimation for weibull distribution, given an array of c shape parameters and the tau_q,
    with the recurrence intervals rec_ints.
    """       
    m = beta.shape[0]
    n = rec_ints.shape[0]

    # log-likelihood    
    log_likelihoods = np.zeros_like(beta)
    
    # precompute matrices for tau_beta and ln_tau_beta
    tau_beta = np.zeros((n, m), dtype=np.float64)
    for i in range(n):
        for j in range(m):
            tau_beta[i, j] = rec_ints[i] / beta[j]
            
    ln_tau_beta = np.log(tau_beta)
    
    c_beta = c / beta
    n_ln_c_beta = n * np.log(c_beta)
    c_1 = c - 1.0

    for j in numba.prange(m):  # no progress indication, it's a parallel for loop
        summ = 0
        
        for i in range(n):
            summ += c_1[j] * ln_tau_beta[i, j] - tau_beta[i, j] ** c[j]
            
        log_likelihoods[j] = n_ln_c_beta[j] + summ
        
    return log_likelihoods

Ora usiamo la funzione per calcolarci il fitting della Weibull per i returns positivi, negativi ed assoluti:

In [ ]:
c = np.arange(0.25, 2, 1e-3)
sfg = sfun.gamma(1.0 + (1.0 / c))

beta = {
    s_type: {
        ret_type: {
            q_type: tau_q[s_type][ret_type][q_type] / sfg
            for q_type in quantile_type
        }
        for ret_type in return_type        
    }
    for s_type in stock_type
}

i_ok = {
    s_type: {
        ret_type: {
            q_type: np.argwhere(beta[s_type][ret_type][q_type] > 1e-6).flatten()
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

beta_ok = {
    s_type: {
        ret_type: {
            q_type: beta[s_type][ret_type][q_type][i_ok[s_type][ret_type][q_type]]
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

c_ok = {
    s_type: {
        ret_type: {
            q_type: c[i_ok[s_type][ret_type][q_type]]
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

In [ ]:
load_log_like = False

ll_weib_file = f"./sp500/log-like-weib_manual_split.pickle"

if load_log_like:
    with open(ll_weib_file, 'rb') as infile:
        log_like_weib = pickle.load(infile)
else:
    log_like_weib = dict()

    for s_type in stock_type:
        log_like_weib[s_type] = dict()
        
        for ret_type in return_type:
            log_like_weib[s_type][ret_type] = dict()
            print(f"\nStock type: {s_type} | Return type: {ret_type}")

            for q_type in quantile_type:
                x = recurrence_intervals[s_type][ret_type][q_type]['n_days'].values
                print(f"Computing Weibull MLE on quantile: {q_type}, c={c_ok[s_type][ret_type][q_type].shape}, beta={beta_ok[s_type][ret_type][q_type].shape}")

                ll = mle_weibull(x, c_ok[s_type][ret_type][q_type], beta_ok[s_type][ret_type][q_type])

                log_like_weib[s_type][ret_type][q_type] = ll

    with open(ll_weib_file, 'wb') as outfile:
          pickle.dump(log_like_weib, outfile)

In [ ]:
colors_mle = {
    'evt': 'lightskyblue',
    '95': 'palegreen',
    '97.5': 'limegreen',
    '99': 'darkgreen',
}

legend_labels_mle = {
    '95': '95%',
    '97.5': '97.5%',
    '99': '99%',
    'evt': 'EVT',
}

titles = {
    'pos': r'Positive $log(r)$',
    'neg': r'Negative $log(r)$',
    'abs': r'Absolute $log(r)$',
}

fig, ax = pl.subplots(nrows=3, ncols=len(stock_type), figsize=(20, 12))

# positive log-returns
for i, ret_type in enumerate(return_type):
    for j, s_type in enumerate(stock_type):
        for q_type in quantile_type:
            ax[i, j].plot(
                c_ok[s_type][ret_type][q_type],
                log_like_weib[s_type][ret_type][q_type],
                color=colors_mle[q_type],
                label=legend_labels_mle[q_type])

            i_max = np.argmax(log_like_weib[s_type][ret_type][q_type])

            ax[i, j].plot(
                c_ok[s_type][ret_type][q_type][i_max],
                log_like_weib[s_type][ret_type][q_type][i_max],
                marker='o',
                color=colors_mle[q_type]
            )

        ax[i, j].set(title=s_type + ' | ' + titles[ret_type], xlabel='c', ylabel=r'$\log(L_W)$')
        ax[i, j].legend(loc='lower right')

sns.despine()

Ok, ora che abbiamo le MLE per i tre tipi di returns e i minimi, possiamo fittare la Weibull sui recurrence intervals:

In [ ]:
i_min = {
    s_type: {
        ret_type: {
            q_type: np.argmax(log_like_weib[s_type][ret_type][q_type])
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

best_shape = {
    'weibull': {
        s_type: {
            ret_type: {
                q_type: c_ok[s_type][ret_type][q_type][i_min[s_type][ret_type][q_type]]
                for q_type in quantile_type
            }
            for ret_type in return_type
        }
        for s_type in stock_type
    }
}

best_scale = {
    'weibull': {
        s_type: {
            ret_type: {
                q_type: tau_q[s_type][ret_type][q_type] / sfun.gamma(1.0 + 1.0 / best_shape['weibull'][s_type][ret_type][q_type])
                for q_type in quantile_type
            }
            for ret_type in return_type
        }
        for s_type in stock_type
    }
}

best_params = {
    'weibull': {
        s_type: {
            ret_type: {
                q_type: {
                    'shape': best_shape['weibull'][s_type][ret_type][q_type],
                    'scale': best_scale['weibull'][s_type][ret_type][q_type],
                    'loc': 0.0,
                }
                for q_type in quantile_type
            }
            for ret_type in return_type
        }
        for s_type in stock_type
    }
}

## 5.2 Fitting della stretched-exponential (s-exp)

Creiamo ora la classe per la s-exp, che ha *pdf*:

$$
p(x, c, a, b) = a e^{-\left( bx \right)^c}
$$

dove $c$, $a$ e $b$ sono *shape parameters*, con $0 < c < 1$, $b \geq 0$ e $a > 0$.

Creo allora la funzione che minimizza la log-likelihood della s-exp, che è

$$
ln(L_{s-exp}) = n \cdot \ln(a) - \sum_{i=1}^{n} (b \cdot x_i)^c
$$

dove $n$ è il numero di recurrence intervals, $a = \frac{c \Gamma \left( \frac{2}{c} \right)}{\left[ \Gamma \left( \frac{1}{c} \right) \right]^2 \tau_Q}$ e $b = \frac{ \Gamma \left( \frac{2}{c} \right)}{\Gamma \left( \frac{1}{c} \right) \tau_Q}$.

In [ ]:
def get_a_b_sexp(c, tau_q):
    """Get the a and b params."""
    gamma_2_c = sfun.gamma(2.0 / c)
    gamma_1_c = sfun.gamma(1.0 / c)
    
    b_all = gamma_2_c / (gamma_1_c * tau_q)
    a_all = b_all * c / gamma_1_c
    
    return a_all, b_all

@timeit
def mle_sexp(rec_ints: np.ndarray, c: np.ndarray, tau_q: float):
    """MLE estimation for s-exponential distribution, given an array of c shape parameters and the tau_q,
    with the recurrence intervals rec_ints.
    """
    n = rec_ints.shape[0]
    
    a_all, b_all = get_a_b_sexp(c, tau_q)
    
    ln_a_all = np.log(a_all)
    
    ll = np.zeros((c.shape[0], ), dtype=np.float64)
    
    for j, c in enumerate(c):
        ssum = 0
        a = a_all[j]
        b = b_all[j]
        
        for i in range(n):
            ssum += np.power((b * rec_ints[i]), c)
            
        ll[j] = n * ln_a_all[j] - ssum
    
    ll[np.isnan(ll)] = -np.inf
        
    return ll

Ora usiamo la funzione per calcolarci il fitting della s-exp per i returns positivi, negativi ed assoluti:

In [ ]:
c_sexp = np.arange(1e-3, 1.0, 1e-3)

In [ ]:
load_log_like = False

ll_sexp_file = f"./sp500/log-like-sexp_manual_split.pickle"

if load_log_like:
    with open(ll_sexp_file, 'rb') as infile:
        log_like_sexp = pickle.load(infile)
else:
    log_like_sexp = dict()

    for s_type in stock_type:
        log_like_sexp[s_type] = dict()
        
        for ret_type in return_type:
            log_like_sexp[s_type][ret_type] = dict()
            print(f"\nReturn type: {ret_type}")

            for q_type in quantile_type:
                x = recurrence_intervals[s_type][ret_type][q_type]['n_days'].values
                print(f"Computing s-exp MLE on quantile: {q_type}, c={c_sexp.shape}")

                ll = mle_sexp(x, c_sexp, tau_q[s_type][ret_type][q_type])

                log_like_sexp[s_type][ret_type][q_type] = ll
            
    log_like_sexp['c'] = c_sexp

    with open(ll_sexp_file, 'wb') as outfile:
          pickle.dump(log_like_sexp, outfile)

Ora prendiamo la massima log-likelihoood:

In [ ]:
i_max_sexp = {
    s_type: {
        ret_type: {
            q_type: np.argmax(log_like_sexp[s_type][ret_type][q_type])
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

best_shape['s-exp'] = {
    s_type: {
        ret_type: {
            q_type: c_sexp[i_max_sexp[s_type][ret_type][q_type]]
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

best_a_sexp = {
    s_type: {
        ret_type: {
            q_type: get_a_b_sexp(best_shape['s-exp'][s_type][ret_type][q_type], tau_q[s_type][ret_type][q_type])[0]
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

best_b_sexp = {
    s_type: {
        ret_type: {
            q_type: get_a_b_sexp(best_shape['s-exp'][s_type][ret_type][q_type], tau_q[s_type][ret_type][q_type])[1]
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

best_params['s-exp'] = {
    s_type: {
        ret_type: {
            q_type: {
                'shape': best_shape['s-exp'][s_type][ret_type][q_type],
                'a': best_a_sexp[s_type][ret_type][q_type],
                'b': best_b_sexp[s_type][ret_type][q_type],
                'loc': 0.0,
            }
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

Plottiamo quindi i risultati della MLE:

In [ ]:
colors_mle = {
    'evt': 'lightskyblue',
    '95': 'palegreen',
    '97.5': 'limegreen',
    '99': 'darkgreen',
}

legend_labels_mle = {
    '95': '95%',
    '97.5': '97.5%',
    '99': '99%',
    'evt': 'EVT',
}

titles = {
    'pos': r'Positive $log(r)$',
    'neg': r'Negative $log(r)$',
    'abs': r'Absolute $log(r)$',
}

fig, ax = pl.subplots(nrows=3, ncols=len(stock_type), figsize=(20, 12))

truncation = 100
for i, ret_type in enumerate(return_type):
    for j, s_type in enumerate(stock_type):
        for q_type in quantile_type:
            ax[i, j].plot(
                c_sexp[truncation:],
                log_like_sexp[s_type][ret_type][q_type][truncation:],
                color=colors_mle[q_type],
                label=legend_labels_mle[q_type])

            ax[i, j].plot(
                c_sexp[i_max_sexp[s_type][ret_type][q_type]],
                log_like_sexp[s_type][ret_type][q_type][i_max_sexp[s_type][ret_type][q_type]],
                marker='o',
                color=colors_mle[q_type]
            )

        ax[i, j].set(title=s_type + ' | ' + titles[ret_type], xlabel='c', ylabel=r'$\log(L_{s-exp})$')
        ax[i, j].legend(loc='lower right')

sns.despine()

## 5.3 Fitting della q-exponential

La terza distribuzione è la [q-exponential](https://en.wikipedia.org/wiki/Q-exponential_distribution).

Creo allora la funzione che minimizza la log-likelihood della q-exp, che è

$$
ln(L_{q-exp}) = n \cdot \ln[\lambda (2 - q)] - \frac{1}{q - 1} \sum_{i=1}^{n} \ln[1 + (q - 1) \lambda \tau_i]
$$

dove $n$ è il numero di recurrence intervals, $\tau_i$ il valore dell'i-esimo recurrence interval, e il parametro $\lambda$ si stima così:

$$
\lambda = \frac{1}{\tau_Q(3 - 2q)}
$$

il parametro libero $q$ ha il range $\left( 0, \frac{3}{2} \right)$, ma noi lo cercheremo nel range $\left( 1, \frac{3}{2} \right)$.

In [ ]:
@timeit
def mle_qexp(rec_ints: np.ndarray, q: np.ndarray, tau_q: float):
    """MLE estimation for q-exponential distribution, given an array of q shape parameters and the tau_q,
    with the recurrence intervals rec_ints.
    """
    assert np.all(q < 1.5)
    
    n = rec_ints.shape[0]
    m = q.shape[0]
    
    lam = 1.0 / (tau_q * (3 - 2 * q))
    
    ll = np.zeros((q.shape[0], ), dtype=np.float64)
    
    for j in range(m):
        ssum = 0
        
        for i in range(n):
            ssum += np.log(1 + (q[j] - 1) * lam[j] * rec_ints[i])
            
        ll[j] = n * np.log(lam[j] * (2 - q[j])) - (1 / (q[j] - 1)) * ssum
        
    return ll

Ora usiamo la funzione per calcolarci il fitting della s-exp per i returns positivi, negativi ed assoluti:

In [ ]:
q_qexp = np.arange(1.0, 1.5, 1e-3)

In [ ]:
load_log_like = False

ll_qexp_file = f"./sp500/log-like-qexp_manual_split.pickle"

if load_log_like:
    with open(ll_qexp_file, 'rb') as infile:
        log_like_qexp = pickle.load(infile)
else:
    log_like_qexp = dict()

    for s_type in stock_type:
        log_like_qexp[s_type] = dict()
        
        for ret_type in return_type:
            log_like_qexp[s_type][ret_type] = dict()
            print(f"\nStock type: {s_type} | Return type: {ret_type}")

            for q_type in quantile_type:
                x = recurrence_intervals[s_type][ret_type][q_type]['n_days'].values
                print(f"Computing q-exp MLE on quantile: {q_type}, c={q_qexp.shape}")

                ll = mle_qexp(x, q_qexp, tau_q[s_type][ret_type][q_type])
                ll[np.isnan(ll)] = -np.inf

                log_like_qexp[s_type][ret_type][q_type] = ll
            
    log_like_qexp['c'] = q_qexp

    with open(ll_qexp_file, 'wb') as outfile:
          pickle.dump(log_like_qexp, outfile)

Ora prendiamo la massima log-likelihoood:

In [ ]:
i_max_qexp = {
    s_type: {
        ret_type: {
            q_type: np.argmax(log_like_qexp[s_type][ret_type][q_type])
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

best_shape['q-exp'] = {
    s_type: {
        ret_type: {
            q_type: q_qexp[i_max_qexp[s_type][ret_type][q_type]]
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

best_lambda_qexp = {
    s_type: {
        ret_type: {
            q_type: 1.0 / (tau_q[s_type][ret_type][q_type] * (3 - 2 * best_shape['q-exp'][s_type][ret_type][q_type]))
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

best_params['q-exp'] = {
    s_type: {
        ret_type: {
            q_type: {
                'shape': best_shape['q-exp'][s_type][ret_type][q_type],
                'lambda': best_lambda_qexp[s_type][ret_type][q_type],
                'loc': 0.0,
            }
            for q_type in quantile_type
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

Plottiamo quindi i risultati della MLE:

In [ ]:
colors_mle = {
    'evt': 'lightskyblue',
    '95': 'palegreen',
    '97.5': 'limegreen',
    '99': 'darkgreen',
}

legend_labels_mle = {
    '95': '95%',
    '97.5': '97.5%',
    '99': '99%',
    'evt': 'EVT',
}

titles = {
    'pos': r'Positive $log(r)$',
    'neg': r'Negative $log(r)$',
    'abs': r'Absolute $log(r)$',
}

fig, ax = pl.subplots(nrows=3, ncols=len(stock_type), figsize=(20, 12))

truncation = 100
for i, ret_type in enumerate(return_type):
    for j, s_type in enumerate(stock_type):
        for q_type in quantile_type:
            ax[i, j].plot(
                q_qexp,
                log_like_qexp[s_type][ret_type][q_type],
                color=colors_mle[q_type],
                label=legend_labels_mle[q_type])

            ax[i, j].plot(
                q_qexp[i_max_qexp[s_type][ret_type][q_type]],
                log_like_qexp[s_type][ret_type][q_type][i_max_qexp[s_type][ret_type][q_type]],
                marker='o',
                color=colors_mle[q_type]
            )

        ax[i, j].set(title=s_type + ' | ' + titles[ret_type], xlabel='c', ylabel=r'$\log(L_{q-exp})$')
        ax[i, j].legend(loc='lower right')

sns.despine()
    
ax[0, 0].set_ylim([-1000, 0])
ax[0, 1].set_ylim([-1000, 0])

ax[1, 0].set_ylim([-1000, 0])
ax[1, 1].set_ylim([-1000, 0])

ax[2, 0].set_ylim([-1500, 0])
ax[2, 1].set_ylim([-1500, 0])

sns.despine()

## 5.4 Calcolo Hazard Probability

Perfetto, ora ho i parametri della Weibull, della s-exp e della q-exp per ogni tipo di return e di threshold. Posso quindi ottenere la curva teorica per il fitting dei recurrence intervals.

Per la Weibull è

$$
W_W(\Delta t | t) = 1 - e^{\left[ \left( \frac{t}{\beta} \right)^\alpha - \left( \frac{t + \Delta t}{\beta} \right)^\alpha \right]}
$$

dove $\alpha = c^*$ lo *shape* ottimale, e $\beta = \frac{\tau_Q}{\Gamma \left( 1 + \frac{1}{\alpha} \right)}$ lo *scale* ottimale.

Per la s-exp è:

$$
W_{s-exp}(\Delta t | t) = \frac{\frac{bc}{a} - \Gamma_l \left( \frac{1}{c}, (bt)^c \right) - \Gamma_u \left( \frac{1}{c}, [b(t + \Delta t)]^c \right)}{\Gamma_u \left( \frac{1}{c}, (bt)^c \right)}
$$

dove

$$
\Gamma_u (a, x) = \Gamma (a, x, +\infty) = \int_{x}^{+\infty} t^{a - 1} e^{-t} dt
$$

è la *upper incomplete Gamma function* e

$$
\Gamma_l (a, x) = \Gamma (a, 0, x) = \int_{0}^{x} t^{a - 1} e^{-t} dt
$$

è la *lower incomplete Gamma function*. Nel nostro caso, abbiamo quindi che $\Gamma_l \left( \frac{1}{c}, (bt)^c \right)$ si traduce in $a = 1/c$ e $x = (bt)^c$, mentre $\Gamma_u \left( \frac{1}{c}, [b(t + \Delta t)]^c \right)$ in $a = 1/c$ e $x = [b(t + \Delta t)]^c$.

Per la q-exp è:

$$
W_{q-exp}(\Delta t | t) = 1 - \left[ 1 + \frac{(q - 1)\lambda \Delta t}{1 + (q - 1)\lambda t} \right]^{1 - \frac{1}{q - 1}}
$$

Mi creo allora le funzioni che le calcolano

In [ ]:
def weibull_hazard(t, shape, scale, delta_t=1):
    part_1 = np.power((t / scale), shape)
    part_2 = np.power(((t + delta_t) / scale), shape)
    
    hazard = 1 - np.exp(part_1 - part_2)
    
    return hazard

def sexp_hazard(t, c, a, b, delta_t=1):
    num1 = (b * c / a)
    num2 = sfun.gammainc((1.0 / c), np.power((b * t), c)) * sfun.gamma(1.0 / c)
    num3 = sfun.gammaincc((1.0 / c), np.power(b * (t + delta_t), c)) * sfun.gamma(1.0 / c)
    
    num = num1 - num2 - num3
    
    denom = sfun.gammaincc((1.0 / c), np.power(b * t, c))
    
    hazard = num / denom
    
    return hazard

def qexp_hazard(t, q, lam, delta_t=1):
    num = (q - 1) * lam * delta_t
    denom = 1 + (q - 1) * lam * t
    exponent = 1 - (1 / (q - 1))
    
    hazard = 1 - np.power((1 + num / denom), exponent)
    
    return hazard

In [ ]:
max_t = 60
x = np.arange(1, max_t + 1)

ret_type = 'neg'
q_type = '99'

th_weibull = {
    s_type: weibull_hazard(
            x,
            best_params['weibull'][s_type][ret_type][q_type]['shape'],
            best_params['weibull'][s_type][ret_type][q_type]['scale']
        )
    for s_type in stock_type
}

th_sexp = {
    s_type: sexp_hazard(
        x,
        best_params['s-exp'][s_type][ret_type][q_type]['shape'],
        best_params['s-exp'][s_type][ret_type][q_type]['a'],
        best_params['s-exp'][s_type][ret_type][q_type]['b'],
    )
    for s_type in stock_type
}

th_qexp = {
    s_type: qexp_hazard(
        x,
        best_params['q-exp'][s_type][ret_type][q_type]['shape'],
        best_params['q-exp'][s_type][ret_type][q_type]['lambda'],
    )
    for s_type in stock_type
}

theoretical_hazard = {
    'weibull': th_weibull,
    's-exp': th_sexp,
    'q-exp': th_qexp,
}

empirical_hazard = {
    s_type: np.array([
        get_empirical_hazard_prob(recurrence_intervals[s_type][ret_type][q_type]['n_days'].values, t, 1)
        for t in x
    ])
    for s_type in stock_type
}

Vediamo com'è il plot della hazard probability empirica e di quella vera per i return negativi al $q_{99}$

In [ ]:
fig, ax = pl.subplots(nrows=1, ncols=2, figsize=(20, 8))

for j, s_type in enumerate(stock_type):
    for dist_type in distribution_type:
        ax[j].plot(
            x,
            theoretical_hazard[dist_type][s_type],
            color=dist_colors[dist_type],
            label=dist_labels[dist_type])

    ax[j].plot(
        x,
        empirical_hazard[s_type],
        label=r'Empirical',
        color='black',
        linestyle='-',
        marker='o',
        markersize=1,
        linewidth=0.5
    )

    ax[j].legend(fontsize=14)
    ax[j].set_xlabel(r'$t$', fontsize=16)
    ax[j].set_ylabel(r'$W(1 | t)$', fontsize=16)
    ax[j].set_title(s_type + ' | ' + r'Hazard prob for $q = 0.99$', fontsize=16)

sns.despine()

# 6. Calcolo del miglior threshold $w_t$ massimizzando la *utility* $U(\theta)$

Ora dobbiamo calcolare il miglior threshold $w_t$ oltre il quale si dà il warning, cioè:

- se $W(1|t) \geq w_t$ --> warning --> 1
- se $W(1|t) < w_t$ --> no warning --> 0

Bisogna definire un peso $\theta$ che si attribuisce alla Recall o al FPR, dove un valore di $\theta$ maggiore dà più peso alla Recall. Inoltre, si definiscono due funzioni:

- la *loss function*, che utilizza il peso:
$$
L(\theta) = \theta (1 - Recall) + (1 - \theta)FPR
$$
- la *utility function*, che dipende dalla *loss*:
$$
U(\theta) = \min(\theta, 1 - \theta) - L(\theta)
$$
che deve essere $U > 0$ per essere utile, e va massimizzata sul *training set*

I passi per farlo sono i seguenti, utilizzando le distribuzioni fittate con i parametri migliori `best_params`:

1. [x] fissare un valore $\hat{\theta}$ per il parametro $\theta$
2. [x] visto che la $Recall = f(w_t)$ e $FPR = g(w_t)$, far variare $w_t \in [0, 1]$ per ottenere tutti i possibili valori di $U(\theta^*)$, cioé:
    1. [x] scegliere un $w_t \in [0, 1] = w_t^i$
    2. [x] calcolare le hazard probability delle tre distribuzioni $W_W(\Delta t|t)$, $W_{s-exp}(\Delta t|t)$ e $W_{q-exp}(\Delta t|t)$ e trasformarle nel target binario $[0, 1]$ a seconda che siano minori o maggiori di $w_t$. Il $\Delta t$ è il periodo tra un intervallo di ricorrenza e l'altro a questo punto
    3. [x] calcolare le metriche $Recall(w_t^i)$, $FPR(w_t^i)$, $KSS(w_t^i)$ dove
    $$
    KSS = Recall - FPR
    $$
    4. [x] calcolare quindi la loss $L(\hat{\theta}, w_t^i) = (1 - Recall(w_t^i))\hat{\theta} + (1 - \hat{\theta})FPR(w_t^i)$
    5. [x] calcolare di conseguenza il valore della utility $U(\hat{\theta})|_{w_t^i}$
3. [x] dopo aver svolto il punto 2 per ogni valore di $w_t \in [0, 1]$, selezionare il massimo $w_t$ con $argmax_{w_t} U(\hat{\theta})$
4. [x] plot della ROC curve per il training ed il testing set
    1. [x] training set
    2. [x] testing set

Cominciamo col definire $\theta$, il range di $w_t$ e una funzione per la recall, l'FPR e il KSS score:

## 6.1 Calcolo scores e performance

In [ ]:
# scelta di theta
theta = 0.5

# w_t
n_points = 1000
w_t = np.linspace(0, 1, n_points + 1)

Ora definisco la *loss* e la *utility*

In [ ]:
def loss_function(theta, recall, fpr):
    """The loss function L = theta * (1 - recall) + (1 - theta) * fpr"""
    assert theta >= 0.0 and theta <= 1.0
    
    return theta * (1 - recall) + (1 - theta) * fpr

def utility_function(theta, loss):
    """The utility function U = min(theta, 1 - theta) - loss"""
    return min(theta, 1 - theta) - loss

e una funzione che, data la funzione di *hazard* teorica, calcola la probabilità in ogni giorno che ci sia un estremo

In [ ]:
# funzione 
def hazard_prob(hazard_fn, extremes: np.ndarray):
    """
    For every time t, predict the hazard probability using the supplied hazard function.
    
    Parameters
    ----------
    hazard_fn: Callable[[int], float]
        function that returns the hazard probability W(1|t), where t is the time passed
        since the last extreme event
    
    extremes: np.ndarray
        binary array of extremes, where 1 means extreme and 0 means normal, tim-ordered
        
    Returns
    -------
    prob: np.ndarray
        hazard probability, at every time t, that there will be an extreme event
    """
    assert isinstance(extremes, np.ndarray)
    
    ext_ind = np.argwhere(extremes == 1).flatten()
    assert ext_ind.shape[0] >= 2  # ci sono almeno 2 estremi, sennò tutto questo non ha senso
    
    n = extremes.shape[0]
    probs = np.zeros((n, ), dtype=np.float64)
    
    # curr_extreme_ind è sempre sull'ultimo estremo visto, a partire dall'inizio, fino al penultimo
    # next_extreme_ind è sempre sul prossimo estremo, fino all'ultimo
    for curr_extreme_ind, next_extreme_ind in zip(ext_ind[:-1], ext_ind[1:]):
        t = 1
        
        while curr_extreme_ind + t <= next_extreme_ind:
            probs[curr_extreme_ind + t] = hazard_fn(t)
            if probs[curr_extreme_ind + t] < 0.0:
                ipdb.set_trace()
            t = t + 1
            
    # ora gli ultimi, può capitare che l'ultimo estremo non sia l'ultimo elemento di extremes
    curr_extreme_ind = ext_ind[-1]
    if curr_extreme_ind < n - 1:
        t = 1
        
        while curr_extreme_ind + t < n:
            probs[curr_extreme_ind + t] = hazard_fn(t)
            t = t + 1
    
    return probs

Bene, ora calcolo le probabilità teoriche per ogni tipo di distribuzione, di return e di quantile.

Ogni volta è la migliore versione per quel tipo di return, di quantile e distribuzione

In [ ]:
def get_all_hazard_probabilities(best_params, extremes, verbose=False):
    """Get all the hazard probabilities, for every distribution, return and threshold type."""
    # per ogni distribuzione, tipo di ritorno e quantile calcola l'hazard probability
    hazard_probabilities = dict()

    for dist_type in distribution_type:
        hazard_probabilities[dist_type] = dict()
        
        for s_type in stock_type:
            hazard_probabilities[dist_type][s_type] = dict()

            for ret_type in return_type:
                hazard_probabilities[dist_type][s_type][ret_type] = dict()

                for q_type in quantile_type:
                    if verbose:
                        print(f"\nDist: {dist_type} | stock: {s_type} | ret_type: {ret_type} | q_type: {q_type}")
                    bp = best_params[dist_type][s_type][ret_type][q_type]
                    ext = extremes[s_type][ret_type][q_type].values

                    if dist_type == 'weibull':
                        shape = bp['shape']
                        scale = bp['scale']

                        if verbose:
                            print(f"Using Weibull with params shape: {shape:4.3f}, scale: {scale:4.3f}")

                        def f(x):
                            return weibull_hazard(x, shape, scale)


                        hazard_probabilities[dist_type][s_type][ret_type][q_type] = hazard_prob(f, ext)

                    elif dist_type == 's-exp':
                        a = bp['a']
                        b = bp['b']
                        c = bp['shape']

                        if verbose:
                            print(f"Using s-exp with params a: {a:4.3f}, b: {b:4.3f}, c: {c:4.3f}")

                        def f(x):
                            return sexp_hazard(x, c, a, b)

                        hazard_probabilities[dist_type][s_type][ret_type][q_type] = hazard_prob(f, ext)

                    elif dist_type == 'q-exp':
                        q = bp['shape']
                        lam = bp['lambda']

                        if verbose:
                            print(f"Using q-exp with params q: {q:4.3f}, lambda: {lam:4.3f}")

                        def f(x):
                            return qexp_hazard(x, q, lam)

                        hazard_probabilities[dist_type][s_type][ret_type][q_type] = hazard_prob(f, ext)
                    else:
                        raise ValueError(f"unrecognized distribution name {dist_type}")
                    
    return hazard_probabilities

In [ ]:
hazard_probabilities = get_all_hazard_probabilities(best_params, extremes)

Verifichiamo che non ci siano elementi negativi (quindi calcoli spurii):

In [ ]:
for dist_type in distribution_type:
    for s_type in stock_type:
        for ret_type in return_type:
            for q_type in quantile_type:
                if np.any(hazard_probabilities[dist_type][s_type][ret_type][q_type] < 0.0):
                    print(f"Errori in dist: {dist_type}, stock: {s_type}, ret_type: {ret_type}, q_type: {q_type}")

                if np.any(hazard_probabilities[dist_type][s_type][ret_type][q_type] > 1.0):
                    print(f"Probabilità > 1 in dist: {dist_type}, stock: {s_type}, ret_type: {ret_type}, q_type: {q_type}")

Ok, prendiamoli come errori numerici.

Ora, a seconda del threshold $w_t$ convertiamo le probabilità in un target binario e calcoliamo le performance, la loss e la utility per ogni valore di $w_t$

In [ ]:
def to_binary(prob: np.ndarray, thresh: float):
    assert thresh <= 1.0 and thresh >= 0.0
    
    return (prob >= thresh).astype(np.int8)


def recall_fpr_kss_precision(y_true, y_pred):
    """Compute recall, fpr and KSS score."""
    tp = np.sum(np.logical_and(y_true, y_pred))
    tn = np.sum(np.logical_and(
        np.logical_not(y_true),
        np.logical_not(y_pred)
    ))
    fp = np.sum(np.logical_and(
        np.logical_not(y_true),
        y_pred
    ))
    fn = np.sum(np.logical_and(
        y_true,
        np.logical_not(y_pred)
    ))
    
    recall = tp / (tp + fn)  # TP / (TP + FN)
    fpr = fp / (fp + tn)  # FP / (FP + TN)
    precision = tp / (tp + fp)
    
    kss = recall - fpr
    
    return recall, fpr, kss, precision

In [ ]:
# per tutti i valori possibili del threshold
def optimize_wt(w, haz_probs, extremes, theta):
    """Find the best threshold and the performance with the supplied w."""
    # devo salvarmi le recall e gli fpr per ogni valore di w, dist, return, quantile
    recalls = {
        dist_type: {
            s_type: {
                ret_type: {
                    q_type: np.zeros((len(w), ), dtype=np.float64)
                    for q_type in quantile_type
                }
                for ret_type in return_type
            }
            for s_type in stock_type
        }
        for dist_type in distribution_type
    }

    fprs = copy.deepcopy(recalls)
    ksss = copy.deepcopy(recalls)
    precisions = copy.deepcopy(recalls)
    losses = copy.deepcopy(recalls)
    utilities = copy.deepcopy(recalls)

    # per tutti i w_t = thresh
    for i, thresh in enumerate(w):
        if i % 200 == 0:
            print(f"iteration: {i}/{len(w)}")
        # per tutte le distribuzioni
        for dist_type in distribution_type:
            
            # per tutte le stock
            for s_type in stock_type:

                # per tutti i tipi di returns
                for ret_type in return_type:

                    # per tutti i tipi di threshold/quantile
                    for q_type in quantile_type:
                        y_pred = to_binary(haz_probs[dist_type][s_type][ret_type][q_type], thresh)
                        y_true = extremes[s_type][ret_type][q_type]

                        recall, fpr, kss, precision = recall_fpr_kss_precision(y_true, y_pred)
                        loss = loss_function(theta, recall, fpr)
                        utility = utility_function(theta, loss)

                        recalls[dist_type][s_type][ret_type][q_type][i] = recall
                        fprs[dist_type][s_type][ret_type][q_type][i] = fpr
                        precisions[dist_type][s_type][ret_type][q_type][i] = precision
                        ksss[dist_type][s_type][ret_type][q_type][i] = kss
                        losses[dist_type][s_type][ret_type][q_type][i] = loss
                        utilities[dist_type][s_type][ret_type][q_type][i] = utility
             
    print("Finished")
    
    # gli indici in w_t dove c'è la combinazione migliore di distribuzione, return e quantili
    best_indexes = {
        dist_type: {
            s_type: {
                ret_type: {
                    q_type: np.argmax(utilities[dist_type][s_type][ret_type][q_type])
                    for q_type in quantile_type
                }
                for ret_type in return_type
            }
            for s_type in stock_type
        }
        for dist_type in distribution_type
    }
    
    best_w = {
        dist_type: {
            s_type: {
                ret_type: {
                    q_type: w[best_indexes[dist_type][s_type][ret_type][q_type]]
                    for q_type in quantile_type
                }
                for ret_type in return_type
            }
            for s_type in stock_type
        }
        for dist_type in distribution_type
    }
    
    return recalls, fprs, ksss, precisions, losses, utilities, best_indexes, best_w

Vediamo le curve ROC per ogni return, threshold e distribuzione

In [ ]:
recalls, fprs, ksss, precisions, losses, utilities, best_indexes, best_w = optimize_wt(w_t, hazard_probabilities, extremes, theta)

## 6.2 Plot delle performance e utilità sul training set

Vediamo allora le curve ROC e la utility per l'azione meno volatile, e poi quella più volatile.

In [ ]:
# meno volatile
size = (7 * len(quantile_type), 7 * len(return_type))
s_type = 'min_vol'

fig, ax = pl.subplots(nrows=len(quantile_type), ncols=(len(ret_type)), figsize=size)
fig.suptitle(f"{s_type} | ROC curves for the training set", fontsize=16)

# sulle righe i quantili
for i, q_type in enumerate(quantile_type):
    
    # sulle colonne i return
    for j, ret_type in enumerate(return_type):
        
        # in ogni grafico, le 3 distribuzioni
        for dist_type in distribution_type:
            i_sorted = np.argsort(fprs[dist_type][s_type][ret_type][q_type])

            x = fprs[dist_type][s_type][ret_type][q_type][i_sorted]
            y = recalls[dist_type][s_type][ret_type][q_type][i_sorted]

            # la ROC
            ax[i, j].plot(
                x,
                y,
                color=dist_colors[dist_type],
                alpha=0.75,
                label=str.title(dist_type),
                marker='.',
                markersize=1.5
            )
            
            # il punto in cui è stato scelto il miglior w_t
            i_sweet = best_indexes[dist_type][s_type][ret_type][q_type]
            best_x = fprs[dist_type][s_type][ret_type][q_type][i_sweet]
            best_y = recalls[dist_type][s_type][ret_type][q_type][i_sweet]
            ax[i, j].plot(
                best_x,
                best_y,
                linestyle='',
                marker='s',
                markersize=5,
                alpha=0.5,
                color=dist_colors[dist_type],
                label=f'best for {str.title(dist_type)}'
            )

        ax[i, j].plot([0, 1], [0, 1], color='black', linewidth=0.5)

        ax[i, j].legend(loc='lower right', fontsize=11)
        
        ax[i, j].set_xlim([0, 1])
        ax[i, j].set_ylim([0, 1])
        
        ax[i, j].set_title(f"returns = {ret_type} | threshold = {q_type}")
        
for a in ax[-1, :]:
    a.set_xlabel('FPR', fontsize=16)
    
for a in ax[:, 0]:
    a.set_ylabel('Recall', fontsize=16)

sns.despine()

Vediamo anche l'utilità, in funzione del threshold

In [ ]:
size = (7 * len(quantile_type), 7 * len(return_type))

fig, ax = pl.subplots(nrows=len(quantile_type), ncols=(len(ret_type)), figsize=size)
fig.suptitle(f"{s_type} | Utility function on the training set", fontsize=16)

x = w_t
# sulle righe i quantili
for i, q_type in enumerate(quantile_type):
    
    # sulle colonne i return
    for j, ret_type in enumerate(return_type):
        
        # in ogni grafico, le 3 distribuzioni
        for dist_type in distribution_type:
            y = utilities[dist_type][s_type][ret_type][q_type]

            ax[i, j].plot(
                x,
                y,
                color=dist_colors[dist_type],
                alpha=0.75,
                label=str.title(dist_type)
            )

        ax[i, j].legend(fontsize=11)
        ax[i, j].set_title(f"returns = {ret_type} | threshold = {q_type}")
        
for a in ax[-1, :]:
    a.set_xlabel(r'$w_t$', fontsize=16)
    
for a in ax[:, 0]:
    a.set_ylabel(r'$U(\theta = {:3.2f})$'.format(theta), fontsize=16)

sns.despine()

E ora l'azione più volatile

In [ ]:
# più volatile
size = (7 * len(quantile_type), 7 * len(return_type))
s_type = 'max_vol'

fig, ax = pl.subplots(nrows=len(quantile_type), ncols=(len(ret_type)), figsize=size)
fig.suptitle(f"{s_type} | ROC curves for the training set", fontsize=16)

# sulle righe i quantili
for i, q_type in enumerate(quantile_type):
    
    # sulle colonne i return
    for j, ret_type in enumerate(return_type):
        
        # in ogni grafico, le 3 distribuzioni
        for dist_type in distribution_type:
            i_sorted = np.argsort(fprs[dist_type][s_type][ret_type][q_type])

            x = fprs[dist_type][s_type][ret_type][q_type][i_sorted]
            y = recalls[dist_type][s_type][ret_type][q_type][i_sorted]

            # la ROC
            ax[i, j].plot(
                x,
                y,
                color=dist_colors[dist_type],
                alpha=0.75,
                label=str.title(dist_type),
                marker='.',
                markersize=1.5
            )
            
            # il punto in cui è stato scelto il miglior w_t
            i_sweet = best_indexes[dist_type][s_type][ret_type][q_type]
            best_x = fprs[dist_type][s_type][ret_type][q_type][i_sweet]
            best_y = recalls[dist_type][s_type][ret_type][q_type][i_sweet]
            ax[i, j].plot(
                best_x,
                best_y,
                linestyle='',
                marker='s',
                markersize=5,
                alpha=0.5,
                color=dist_colors[dist_type],
                label=f'best for {str.title(dist_type)}'
            )

        ax[i, j].plot([0, 1], [0, 1], color='black', linewidth=0.5)

        ax[i, j].legend(loc='lower right', fontsize=11)
        
        ax[i, j].set_xlim([0, 1])
        ax[i, j].set_ylim([0, 1])
        
        ax[i, j].set_title(f"returns = {ret_type} | threshold = {q_type}")
        
for a in ax[-1, :]:
    a.set_xlabel('FPR', fontsize=16)
    
for a in ax[:, 0]:
    a.set_ylabel('Recall', fontsize=16)

sns.despine()

Vediamo anche l'utilità, in funzione del threshold

In [ ]:
size = (7 * len(quantile_type), 7 * len(return_type))

fig, ax = pl.subplots(nrows=len(quantile_type), ncols=(len(ret_type)), figsize=size)
fig.suptitle(f"{s_type} | Utility function on the training set", fontsize=16)

x = w_t
# sulle righe i quantili
for i, q_type in enumerate(quantile_type):
    
    # sulle colonne i return
    for j, ret_type in enumerate(return_type):
        
        # in ogni grafico, le 3 distribuzioni
        for dist_type in distribution_type:
            y = utilities[dist_type][s_type][ret_type][q_type]

            ax[i, j].plot(
                x,
                y,
                color=dist_colors[dist_type],
                alpha=0.75,
                label=str.title(dist_type)
            )

        ax[i, j].legend(fontsize=11)
        ax[i, j].set_title(f"returns = {ret_type} | threshold = {q_type}")
        
for a in ax[-1, :]:
    a.set_xlabel(r'$w_t$', fontsize=16)
    
for a in ax[:, 0]:
    a.set_ylabel(r'$U(\theta = {:3.2f})$'.format(theta), fontsize=16)

sns.despine()

## 7. Predizione nel periodo out-of-sample (test set)

Ora bisogna utilizzare le 3 distribuzioni, già fittate sul training set, per predire sul test set.

Riutilizzo il codice appena usato:

In [ ]:
hazard_probabilities_test = get_all_hazard_probabilities(best_params, extremes_test)

In [ ]:
def predict_and_evaluate(hazard_probs, extremes, best_w, theta):
    """Predict the classes and evaluate performance of the models.
    best_w is the best w threshold, for every dist, return and quantile.
    """
    # devo salvarmi le recall e gli fpr per ogni valore di w, dist, return, quantile
    recalls = {
        dist_type: {
            s_type: {
                ret_type: {
                    q_type: 0.0
                    for q_type in quantile_type
                }
                for ret_type in return_type
            }
            for s_type in stock_type
        }
        for dist_type in distribution_type
    }

    fprs = copy.deepcopy(recalls)
    ksss = copy.deepcopy(recalls)
    precisions = copy.deepcopy(recalls)
    losses = copy.deepcopy(recalls)
    utilities = copy.deepcopy(recalls)

    # per tutte le distribuzioni
    for dist_type in distribution_type:
        
        # per tutte le azioni
        for s_type in stock_type:

            # per tutti i tipi di returns
            for ret_type in return_type:

                # per tutti i tipi di threshold/quantile
                for q_type in quantile_type:
                    y_pred = to_binary(
                        hazard_probs[dist_type][s_type][ret_type][q_type],
                        best_w[dist_type][s_type][ret_type][q_type]
                    )
                    y_true = extremes[s_type][ret_type][q_type]

                    recall, fpr, kss, precision = recall_fpr_kss_precision(y_true, y_pred)
                    loss = loss_function(theta, recall, fpr)
                    utility = utility_function(theta, loss)

                    recalls[dist_type][s_type][ret_type][q_type] = recall
                    fprs[dist_type][s_type][ret_type][q_type] = fpr
                    ksss[dist_type][s_type][ret_type][q_type] = kss
                    precisions[dist_type][s_type][ret_type][q_type] = precision
                    losses[dist_type][s_type][ret_type][q_type] = loss
                    utilities[dist_type][s_type][ret_type][q_type] = utility

    return recalls, fprs, ksss, precisions, losses, utilities

In [ ]:
recalls_test, fprs_test, ksss_test, precisions_test, losses_test, utilities_test = predict_and_evaluate(hazard_probabilities_test, extremes_test, best_w, theta)

## 8. Confronto in-sample vs out-of-sample

Confrontiamo allora le performance su training e test set.

Bisogna tenere a mente, però, che il training set ed il testing set condividono parte delle due crisi (finanziaria e debito EU), e quindi le performance potrebbero essere addirittura migliori sul testing set.

### 8.1 Plot di $W(1|t)$ e $W_{emp}(1|t)$

Vediamo le hazard probability empiriche e fittate, sia sul training che sul testing set, coi returns negativi e threshold 99%.

In [ ]:
max_t = 60
x = np.arange(1, max_t + 1)

ret_type = 'neg'
q_type = '99'
theoretical_hazard_test = dict()

# per ogni distribuzione
for dist_type in distribution_type:
    theoretical_hazard_test[dist_type] = dict()
    
    # per ogni stock
    for s_type in stock_type:
        if dist_type == 'weibull':
            theoretical_hazard_test[dist_type][s_type] = weibull_hazard(
                x,
                best_params[dist_type][s_type][ret_type][q_type]['shape'],
                best_params[dist_type][s_type][ret_type][q_type]['scale'],
            )
        elif dist_type == 's-exp':
            theoretical_hazard_test[dist_type][s_type] = sexp_hazard(
                x,
                best_params[dist_type][s_type][ret_type][q_type]['shape'],
                best_params[dist_type][s_type][ret_type][q_type]['a'],
                best_params[dist_type][s_type][ret_type][q_type]['b'],
            )
        elif dist_type == 'q-exp':
            theoretical_hazard_test[dist_type][s_type] = qexp_hazard(
                x,
                best_params[dist_type][s_type][ret_type][q_type]['shape'],
                best_params[dist_type][s_type][ret_type][q_type]['lambda'],
            )

empirical_hazard_test = {
    s_type: np.array([
        get_empirical_hazard_prob(recurrence_intervals_test[s_type][ret_type][q_type]['n_days'].values, t, 1)
        for t in x
    ])
    for s_type in stock_type
}

In [ ]:
# sulle righe train vs test, sulle colonne le due azioni
fig, ax = pl.subplots(nrows=2, ncols=2, figsize=(20, 10))

dist_colors = {
    'weibull': 'orchid',
    's-exp': 'orangered',
    'q-exp': 'mediumblue'
}

dist_labels = {
    'weibull': r'Weibull',
    's-exp': r's-exp',
    'q-exp': r'q-exp',
}

datasets = ['training', 'testing']

for j, s_type in enumerate(stock_type):
    for i, dataset in enumerate(datasets):
        if i == 0:
            y_emp = empirical_hazard[s_type]
        elif i == 1:
            y_emp = empirical_hazard_test[s_type]
        
        for dist_type in distribution_type:
            if i == 0:
                y = theoretical_hazard[dist_type][s_type]
            elif i == 1:
                y = theoretical_hazard_test[dist_type][s_type]
                
            ax[i, j].plot(
                x,
                y,
                color=dist_colors[dist_type],
                label=dist_labels[dist_type]
            )
            
        ax[i, j].plot(
            x,
            y_emp,
            label=r'Empirical',
            color='black',
            linestyle='-',
            marker='o',
            markersize=1,
            linewidth=0.5
        )
        
        ax[i, j].legend(fontsize=14)
        ax[i, j].set_xlabel(r'$t$', fontsize=16)
        ax[i, j].set_ylabel(r'$W(1 | t)$', fontsize=16)
        ax[i, j].set_title(f'Hazard prob for q99 | {s_type} | {ret_type} returns | {dataset}', fontsize=16)

sns.despine()

### 8.2 Recall, FPR, KSS score, Precision al variare di $w_t$

Vediamo ora come i valori di performance sul training e testing set, per il threshold ottimale $w_t^*$.

Prima calcolo i risultati ottenuti con il threshold migliore sul training set:

In [ ]:
#-----------------------------------#
# recall per w_t ottimale
best_recalls = {
    dist_type: {
        s_type: {
            ret_type: {
                q_type: recalls[dist_type][s_type][ret_type][q_type][best_indexes[dist_type][s_type][ret_type][q_type]]
                for q_type in quantile_type
            }
            for ret_type in return_type
        }
        for s_type in stock_type
    }
    for dist_type in distribution_type
}

#-----------------------------------#
# fpr per w_t ottimale
best_fprs = {
    dist_type: {
        s_type: {
            ret_type: {
                q_type: fprs[dist_type][s_type][ret_type][q_type][best_indexes[dist_type][s_type][ret_type][q_type]]
                for q_type in quantile_type
            }
            for ret_type in return_type
        }
        for s_type in stock_type
    }
    for dist_type in distribution_type
}

#-----------------------------------#
# kss per w_t ottimale
best_ksss = {
    dist_type: {
        s_type: {
            ret_type: {
                q_type: ksss[dist_type][s_type][ret_type][q_type][best_indexes[dist_type][s_type][ret_type][q_type]]
                for q_type in quantile_type
            }
            for ret_type in return_type
        }
        for s_type in stock_type
    }
    for dist_type in distribution_type
}

#-----------------------------------#
# precision per w_t ottimale
best_precisions = {
    dist_type: {
        s_type: {
            ret_type: {
                q_type: precisions[dist_type][s_type][ret_type][q_type][best_indexes[dist_type][s_type][ret_type][q_type]]
                for q_type in quantile_type
            }
            for ret_type in return_type
        }
        for s_type in stock_type
    }
    for dist_type in distribution_type
}

#-----------------------------------#
# utility per w_t ottimale
best_utilities = {
    dist_type: {
        s_type: {
            ret_type: {
                q_type: utilities[dist_type][s_type][ret_type][q_type][best_indexes[dist_type][s_type][ret_type][q_type]]
                for q_type in quantile_type
            }
            for ret_type in return_type
        }
        for s_type in stock_type
    }
    for dist_type in distribution_type
}

Poi mi creo una funzione che crei la tabella, per una singola distribuzione, per un singolo tipo di return:

In [ ]:
def get_results_table(train_performance, test_performance, dist_type: str, s_type: str, ret_type: str):
    """Get a table similar to those on page 366 of the paper.
    
    Parameters
    ----------
    train_performance: List[Dict]
        performance on the train set, list of dicts with order 
        - recall
        - fpr
        - kss
        - precision
        - utility
        each of which is a tree with first-level keys the return_type,
        second-level keys the quantile_type
    
    test_performance: List[Dict]
        performance on the test set, list of dicts with order 
        - recall
        - fpr
        - kss
        - precision
        - utility
        each of which is a tree with first-level keys the return_type,
        second-level keys the quantile_type
        
    Returns
    -------
    table: pd.DataFrame
        a DataFrame with quantile_type on the columns, and rows
        - fpr (in/out)
        - recall (in/out)
        - utility (in/out)
        - kss (in/out)
    """
    rec_train, fpr_train, kss_train, prec_train, u_train = train_performance
    rec_test, fpr_test, kss_test, prec_test, u_test = test_performance
    
    columns = quantile_type
    index = pd.Index([
        'in: FPR', 'out: FPR', 'in: Recall', 'out: Recall',
        'in: Prec', 'out: Prec',
        'in: U', 'out: U', 'in: KSS', 'out: KSS',
    ], name=dist_type)
    
    table = pd.DataFrame(
        data=np.zeros((len(index), len(columns)), dtype=np.float64),
        columns=columns,
        index=index
    )
    
    for q_type in quantile_type:
        table.loc['in: FPR', q_type] = fpr_train[dist_type][s_type][ret_type][q_type]
        table.loc['out: FPR', q_type] = fpr_test[dist_type][s_type][ret_type][q_type]
        
        table.loc['in: Recall', q_type] = rec_train[dist_type][s_type][ret_type][q_type]
        table.loc['out: Recall', q_type] = rec_test[dist_type][s_type][ret_type][q_type]
        
        table.loc['in: Prec', q_type] = prec_train[dist_type][s_type][ret_type][q_type]
        table.loc['out: Prec', q_type] = prec_test[dist_type][s_type][ret_type][q_type]
        
        table.loc['in: U', q_type] = u_train[dist_type][s_type][ret_type][q_type]
        table.loc['out: U', q_type] = u_test[dist_type][s_type][ret_type][q_type]
        
        table.loc['in: KSS', q_type] = kss_train[dist_type][s_type][ret_type][q_type]
        table.loc['out: KSS', q_type] = kss_test[dist_type][s_type][ret_type][q_type]
    
    return table

e la uso per calcolare le tabelle per ogni distribuzione.

Infine, concateno le tabelle per lo stesso tipo di return per ottenere le tabelle multi-dimensionali di pagina 366 del paper:

In [ ]:
performance_train = {
    dist_type: {
        s_type: (
            best_recalls[dist_type][s_type],
            best_fprs[dist_type][s_type],
            best_ksss[dist_type][s_type],
            best_precisions[dist_type][s_type],
            best_utilities[dist_type][s_type]
        )
        for s_type in stock_type
    }
    for dist_type in distribution_type
}

performance_test = {
    dist_type: {
        s_type: (
            recalls_test[dist_type][s_type],
            fprs_test[dist_type][s_type],
            ksss_test[dist_type][s_type],
            precisions_test[dist_type][s_type],
            utilities_test[dist_type][s_type]
        )
        for s_type in stock_type
    }
    for dist_type in distribution_type
}

performance_train = (
    best_recalls,
    best_fprs,
    best_ksss,
    best_precisions,
    best_utilities,
)

performance_test = (
    recalls_test,
    fprs_test,
    ksss_test,
    precisions_test,
    utilities_test,
)

result_tables = {
    dist_type: {
        s_type: {
            ret_type: get_results_table(
                performance_train,
                performance_test,
                dist_type,
                s_type,
                ret_type
            )
            for ret_type in return_type
        }
        for s_type in stock_type
    }
    for dist_type in distribution_type
}

result_panels = {
    s_type: {
        ret_type: pd.concat([
                result_tables[dist_type][s_type][ret_type] for dist_type in distribution_type
            ],
            axis='columns',
            keys=distribution_type
        )
        for ret_type in return_type
    }
    for s_type in stock_type
}

## 9. Conclusioni

Abbiamo replicato il paper, stavolta sulle azioni S&P500, dove abbiamo preso due azioni, la meno volatile e la più volatile, dal 2005 ad oggi.

Il criterio di inclusione in training o testing set è stato quello di includere nel training almeno metà delle due crisi successive al 2005, quella finanziaria del 2007-2008 e quella del debito dell'eurozona negli anni 2010-2014.

È stata svolta una ottimizzazione per la ricerca del threshold ottimale per la *hazard probability* $W(\Delta t | t)$, e trasformato quindi il problema in un problema di classificazione.

Vediamo i risultati di questa analisi, cominciando dalla meno volatile.

### 9.1 Azione meno volatile

Vediamo i returns negativi per primi:

In [ ]:
result_panels['min_vol']['neg']

Vediamo che la recall è molto alta sia sul training che sul testing set, mentre la precision è molto bassa, attorno al 10%, per tutte e tre le distribuzioni. Si vede che il threshold della EVT non è adeguato.

Vediamo allora sui returns positivi:

In [ ]:
result_panels['min_vol']['pos']

Anche qui, idem.

Vediamo i returns assoluti:

In [ ]:
result_panels['min_vol']['abs']

Lievemente più alte la precision per il quantile 95 e 97.5, ma comunque basse basse.

### 9.2 Azione più volatile

Vediamo come è andata sull'azione più volatile. Cominciamo anche qui dai returns negativi:

In [ ]:
result_panels['max_vol']['neg']

Le performance sono migliori, con una precision che oscilla tra il 10 ed il 22%, del tutto comparabile agli esperimenti con rete neurale che abbiamo già fatto. La Recall resta molto alta, attorno all'87%.

Vediamo i returns positivi:

In [ ]:
result_panels['max_vol']['pos']

Idem, ma sul $q_{95}$ si raggiunge una precision del 25%.

Infine quelli assoluti:

In [ ]:
result_panels['max_vol']['abs']

Anche qui idem, si tocca un massimo di 26% di recall.

### 9.3 Plot precision-recall per le due azioni

In [ ]:
# sulle colonne le azioni, sulle righe le distribuzioni
fig, ax = pl.subplots(nrows=3, ncols=2, figsize=(20, 15))

for j, s_type in enumerate(stock_type):
    for i, dist_type in enumerate(distribution_type):
        ax[i, j].scatter(
            precisions[dist_type][s_type]['neg']['95'],
            recalls[dist_type][s_type]['neg']['95'],
            edgecolors='white',
            color=dist_colors[dist_type],
            alpha=0.7
        )

        ax[i, j].set_title(f"{str.title(dist_type)} | {s_type}", fontsize=16)
        ax[i, j].set_xlim([0, 1.05])
        ax[i, j].set_ylim([0, 1.05])
    
ax[-1, 0].set_xlabel('Precision', fontsize=14)
ax[-1, 1].set_xlabel('Precision', fontsize=14)
ax[0, 0].set_ylabel('Recall', fontsize=14)
ax[1, 0].set_ylabel('Recall', fontsize=14)
ax[2, 0].set_ylabel('Recall', fontsize=14)
sns.despine()

Questo riflette esattamente lo stesso problema avuto con i primi esperimenti di Deep Learning.

Tuttavia, in questo caso, si può agire sul threshold $\theta$ della *loss function*, che va a pesare il ratio dei falsi allarmi rispetto a alla recall.
In questo modo, si può privilegiare un più basso tasso di falsi allarmi e perdere qualche evento estremo.

Le performance serviranno da base per il lavoro con il nuovo modello di Deep Learning.

## COSA NON FUNZIONA

La EVT non dà un buon threshold, anzi finisce sotto la media dei corrispondenti returns. Purtroppo non so cosa farci.